# AimBig Star Ratings Prediction 

### Extracting the Data - for September 2022 Star Rating (1)
- From 1st Star Rating assessment period (for the Sept 2022 Star Rating)
- Assessment period: 1/10/20-7/10/22 (8 quarters)

In [1]:
#Packages:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


- Need to split DMS, ESS -> to apply performance weightings

### To-Do 20/09 
- Subset each of the dataframes to include attributes needed for star rating calculation - done
- Join dataframes to do with regression analysis -> compare attribute values with the "JSCI Personal Factors Outcome"
- Get the performance measure (current star ratings report) - and filter for the data in the 1st star rating period - done
- Compare star rating calculation with filtered data. - in progress

### Analysing from Participant -> Site -> Contract Level 


## Step 1) Actual Performance vs. Expected Performance 
Performance: numerator/denominator for caseload per contract
- Consider DMS and ESS contracts separately
- Only focusing on full outcomes (13, 26, 52 week full outcomes)

- Calculating the performance (actual vs expected) every quarter 

Placement_status: if participant has commenced employment
Placement_Phase: PPS: post-placement-support

Quarterisation:
- Look at example from government pdf (if a person doesn't achieve) - Figure 1
- Check which records is included for a participant 
- Pre-quarterisation: Looking at a participants overall period with a contract (from start date -> end date)
    - This period doesn't consider the start date to end date (broken up by quarters)
- Post-quarterisation: Breaking up the pre-quarterisation period (from start date - to end date) by quarters. Looking at whether a participant has achieved an outcome in each quarter (within the start-end date). If they haven't, then the denominator = 1, and overall denominator += 1. 


## TO-DO 27/09-28/09:
1. Pre-quarterisation periods - start pre-quarterisation period analysis.
    1.1 Overview of participants with a contract (start date -> end date) - done
    
    1.2 LifeCycle File: Investigate the job_seeker_id, placement_status_code, placement_phase_code, placement_sequence_number. The dates: Referral_Date, Commencement_Date, Time_In_Program_Weeks - done

    1.3 From 1.1 - Investigate the participants who numerate/denominate. - Using claims file (after joining to the two other files)
        - Split data into 13, 26, 52 FULL outcomes (Performance measures)
        - Records not included in numerator or denominator: record is not included, as the participant time to achieve an outcome is less than 13/26/52 weeks (check commencement date/outcome evaluation date)

    1.4 Compare pre-quarterisation period with expected performance (calculated by Arriba Group)
    
    1.5 Join job_seeker_id to caseload file. - done
    


- After each quarter, the placement status code is recorded. It is sustained if there are no changes to the status code since the previous quarter. 


Working: 
- Need to be still in assistance at the end of the two year rolling period.
COM_EA: commenced employment with assistance
COM_PPS: commenced employment with post placement support 
COM_OS: commenced employment with ongoing support 
TRN_PPS: transferred with post placement support 
TRN_EA: transferred with employment assistance



- Include these outcomes - given the date period

Exited:
- Exited the site during the two year rolling period
- Include both of these:
EXT_EA: exited the site with assistance, or exited assistance
EXT_PPS: exited the site with post placement support 

exited the Site prior to the two year rolling period but have an associated Outcome which was lodged during the two year period at the Site; or
- Get this from claims 
- Note: this won't be from lifecycle file

exited prior to the two year rolling period but had an ongoing support employment verification date which is within the two year rolling period.
Variables: OSA Date, OSA Due Date (needs to be in the 2 year rolling period)
- don't denominate or numerate

SUS_EA: suspended with employment assistance (Counted in denominator if completed more than 13 weeks with us. Includes suspension period)


### Pathway Meaning
- Include a table with numerator/denominator boolean columns -> initial plan
- Determining whether a combination should be included in a numerator/denominator or both

### Contract Level - 13, 26, 52 outcomes: participants performance on a contract level

## Claims and Recording Outcomes
- The COMMENCEMENT_DATE column: marks the date from which a participant is expected to record an outcome. 
- Numerates: 13/26/52 weeks from COMMENCEMENT_DATE - an outcome is achieved.
- COMMENCEMENT_DATE changes based on the PLACEMENT_SEQUENCE_NUMBER.
- Participant needs to numerate a 13wk full outcome before numerating a 26wk full outcome.
- OUTCOME_TYPE: employment only
- Don't count the service fee.
- Edge case: If no claim is made 13wks after the COMMENCEMENT_DATE, but a 26wk claim is made after 13wk - sometimes 13wk claim is made on the same date (late).
- Service fee claims are not counted in numerator
- We get outcome claims when participant is in the PPS phase.
- Once a participant completes 26 weeks - they denominate in 26 weeks, etc.
- Calculating num/denom separately for each full outcome type


In [2]:
## Loading the comb df in:
df_comb =  pd.read_csv('C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/apv_stream_participant_versions.csv') # reading from github repo

df_comb = df_comb.rename(columns={'Exit_Date' : 'EXIT_DATE', 'placement_type' : 'PLACEMENT_TYPE'})




In [3]:
# drop rows where COMMENCEMENT_DATE_NEW is NULL -> no valid placements
df_comb = df_comb[df_comb['COMMENCEMENT_DATE'].isna() == False]
# drop None
df_comb = df_comb[df_comb['COMMENCEMENT_DATE'] != None]

In [4]:
df_comb.columns

Index(['COMMENCEMENT_DATE', 'Program_Type_Code', 'jr_jobseeker_id',
       'Job_Seeker_ID', 'plac_jobseeker_ID', 'PLACEMENT_TYPE', 'Program',
       'Contract_ID', 'Site_Code', 'Site_Name', 'ESA_Code', 'ESA Name',
       'State', 'specialist_site_type_code', 'Referral_Date',
       'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE', 'Phase_changed',
       'Status_changed', 'Phase_commencement_date', 'Time_In_Program_Weeks',
       'Period_of_Service_Weeks', 'Disability_Type',
       'moderate_intellectual_disability', 'indigenous', 'volunteer',
       'jsci_status', 'employment_benchmark', 'esat_jca_assessment_date',
       'EXIT_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE',
       'CLAIM_AVAILABLE_DATE', 'Claim_Rate_Type', 'Claim_Status', 'Claim_ID',
       'Claim_Type', 'OUTCOME_TYPE', 'CLAIM_START_DATE', 'CLAIM_END_DATE',
       'overridden_outcome_started_on', 'pps_start_date'],
      dtype='object')

In [5]:
df_comb

,COMMENCEMENT_DATE,Program_Type_Code,jr_jobseeker_id,Job_Seeker_ID,plac_jobseeker_ID,PLACEMENT_TYPE,Program,Contract_ID,Site_Code,Site_Name,...,CLAIM_AVAILABLE_DATE,Claim_Rate_Type,Claim_Status,Claim_ID,Claim_Type,OUTCOME_TYPE,CLAIM_START_DATE,CLAIM_END_DATE,overridden_outcome_started_on,pps_start_date
0,2021-07-29 00:00:00,DMS,2941,2738240909,2941,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,...,NaN,NaN,NaN,77340,Des week 52,NaN,NaN,NaN,NaN,2021-07-29
1,2021-07-29 00:00:00,DMS,2941,2738240909,2941,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,...,NaN,NaN,NaN,77339,Des week 26,NaN,NaN,NaN,NaN,2021-07-29
2,2021-07-29 00:00:00,DMS,2941,2738240909,2941,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,...,NaN,NaN,NaN,63058,Des week 52,NaN,NaN,NaN,NaN,2021-07-29
3,2021-07-29 00:00:00,DMS,2941,2738240909,2941,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,...,NaN,DES 4wk Outcome,Approved,63055,Des week 4,Employment,2021-07-19 00:00:00,2021-08-15 00:00:00,NaN,2021-07-29
4,2021-07-29 00:00:00,DMS,2941,2738240909,2941,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,...,NaN,NaN,NaN,63057,Des week 26,NaN,NaN,NaN,NaN,2021-07-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461806,2022-02-16 00:00:00,DMS,1437,450740,1437,Vacancy Placement,DMS,0212859D,ML67,AimBig Employment BALLARAT CENTRAL,...,NaN,DES 4wk Outcome,Approved,81482,Des week 4,Employment,2021-12-23 00:00:00,2022-01-19 00:00:00,NaN,2022-02-16
461807,2022-02-16 00:00:00,DMS,1437,450740,1437,Vacancy Placement,DMS,0212859D,ML67,AimBig Employment BALLARAT CENTRAL,...,NaN,NaN,NaN,81485,Des week 52,NaN,NaN,NaN,NaN,2022-02-16
461808,2021-05-27 00:00:00,ESS,10664,8390637409,10664,Vacancy Placement,ESS,0212903C,AA01,AimBig Employment PENRITH,...,NaN,NaN,NaN,57294,Des week 26,NaN,NaN,NaN,NaN,2021-05-27
461809,2021-05-27 00:00:00,ESS,10664,8390637409,10664,Vacancy Placement,ESS,0212903C,AA01,AimBig Employment PENRITH,...,NaN,NaN,NaN,57292,Des week 4,NaN,NaN,NaN,NaN,2021-05-27


In [6]:
len(df_comb)

391633

In [7]:
# Assigning it to df_pre
df_pre = df_comb[["Job_Seeker_ID", 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE', 'PLACEMENT_TYPE', 'Program',
       'Contract_ID', 'Site_Code', 'Site_Name', 'ESA_Code', 'ESA Name',
       'State', 'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE', 'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE']]
df_pre = df_pre.sort_values(by=['Job_Seeker_ID', 'CLAIM_START_DATE'])

# converting data types + converting NaT to out of range dates (i.e. 2001)
df_pre['Job_Seeker_ID'] = pd.to_numeric(df_pre['Job_Seeker_ID'], errors='coerce')
df_pre = df_pre[df_pre['Job_Seeker_ID'].isna() != True]
df_pre['Job_Seeker_ID'] = df_pre['Job_Seeker_ID'].astype('int64')


# need to convert column into string first -> then back to date
dict_types = {'SUSPENSION_FROM_DATE' : 'str', 'COMMENCEMENT_DATE' : 'str', 'CLAIM_START_DATE' : 'str', 'CLAIM_AVAILABLE_DATE' : 'str', 'CLAIM_END_DATE' : 'str', 'SUSPENSION_ACTUAL_END_DATE' : 'str' }
df_pre = df_pre.astype(dict_types)

# fill NA values with out of bounds date
fillVal = pd.to_datetime('2001-01-01', format='%Y-%m-%d')




df_pre['SUSPENSION_FROM_DATE'] = pd.to_datetime(df_pre['SUSPENSION_FROM_DATE'], format='%Y-%m-%d')
df_pre['SUSPENSION_FROM_DATE'] = df_pre['SUSPENSION_FROM_DATE'].fillna(fillVal)


df_pre['SUSPENSION_ACTUAL_END_DATE'] = pd.to_datetime(df_pre['SUSPENSION_ACTUAL_END_DATE'], format='%Y-%m-%d')
df_pre['SUSPENSION_ACTUAL_END_DATE'] = df_pre['SUSPENSION_ACTUAL_END_DATE'].fillna(fillVal)

df_pre['COMMENCEMENT_DATE'] = pd.to_datetime(df_pre['COMMENCEMENT_DATE'], format='%Y-%m-%d')
df_pre['COMMENCEMENT_DATE'] = df_pre['COMMENCEMENT_DATE'].fillna(fillVal)


df_pre['CLAIM_START_DATE'] = pd.to_datetime(df_pre['CLAIM_START_DATE'], format='%Y-%m-%d')
df_pre['CLAIM_START_DATE'] = df_pre['CLAIM_START_DATE'].fillna(fillVal)

df_pre['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_pre['CLAIM_AVAILABLE_DATE'], format='%Y-%m-%d')
df_pre['CLAIM_AVAILABLE_DATE'] = df_pre['CLAIM_AVAILABLE_DATE'].fillna(fillVal)

df_pre['CLAIM_END_DATE'] = pd.to_datetime(df_pre['CLAIM_END_DATE'], format='%Y-%m-%d', errors='coerce')
df_pre['CLAIM_END_DATE'] = df_pre['CLAIM_END_DATE'].fillna(fillVal)




1/10/20-7/10/22

In [8]:
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')



df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].replace(np.nan, 'na', regex=True)
df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].replace('', 'na', regex=True)
df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].astype('str')



# remove duplicates - make copy of duplicate free dataframe 
df_pre = df_pre.loc[:,~df_pre.columns.duplicated()].copy()




### Performance - Numerator
- Aggregate numerator per job_seeker_id
- aggregate numerator for each type of full outcome
- Decide what to do for denominators..

## Denominators
- Duration at assistance is at least 13/26/52 weeks (excluding suspension periods)
- Excludes participants who have achieved a pathway outcome

In [9]:
# checking suspensions or early exits -> adding 13, 26, 52 weeks to the commencement date
week_13 = dt.timedelta(days = 91)
week_26 = dt.timedelta(days = 182)
week_52 = dt.timedelta(weeks = 364)


# getting the period between suspension_from_date and suspension_actual_end_date in weeks
df_pre['SUSPENSION_DAYS'] = (df_pre['SUSPENSION_ACTUAL_END_DATE'] - df_pre['SUSPENSION_FROM_DATE']).apply(lambda x: x/np.timedelta64(1, 'D'))

df_pre['SUSPENSION_DAYS'] = df_pre['SUSPENSION_DAYS'].astype('int64')

df_pre.head(10)



,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Code,Site_Name,ESA_Code,ESA Name,...,Claim_ID,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS
5744,44710,EA,Commenced,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90921,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
5745,44710,EA,Commenced,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90922,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
5746,44710,EA,Commenced,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90919,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
5747,44710,EA,Commenced,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90920,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
44471,44710,EA,Suspended,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90921,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
44472,44710,EA,Suspended,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90922,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
44473,44710,EA,Suspended,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90919,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
44474,44710,EA,Suspended,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90920,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
65388,44710,EA,Commenced,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90921,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4
65389,44710,EA,Commenced,Vacancy Placement,Ess3,0212879H,MK77,AimBig Employment TOOWONG,4NOB,QLD North,...,90922,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-03-02,2023-03-06,NaN,4


In [10]:
df_pre['REFERRAL_PHASE_CODE'].drop_duplicates()

5744       EA
66575     PPS
154872     OS
279868     NE
52581     NaN
Name: REFERRAL_PHASE_CODE, dtype: object

In [11]:
# for denominator: include working codes 
# since referral phase code can only be PPS or EA
ref_pps = (df_pre["REFERRAL_PHASE_CODE"] == 'PPS')
ref_ea = (df_pre["REFERRAL_PHASE_CODE"] == 'EA')
ref_os = (df_pre["REFERRAL_PHASE_CODE"] == 'OS')
com = (df_pre["PLACEMENT_STATUS_CODE"] == 'Commenced')
trn = (df_pre["PLACEMENT_STATUS_CODE"] == 'Transferred')

com_ea = com & ref_ea
com_pps = com & ref_pps
com_os = com & ref_os 
trn_pps = trn & ref_pps
trn_ea = trn & ref_ea 


# include exit codes - but check date (exited during 2 year rolling period): 
# just do start of 2018
date_range = df_pre['COMMENCEMENT_DATE'].loc[(df_pre['COMMENCEMENT_DATE'] >= '2020-10-01') & (df_pre['COMMENCEMENT_DATE'] <= '2022-10-07')]


ext = (df_pre["REFERRAL_PHASE_CODE"] == 'Exited')
#ext_ea = (ext & ref_ea) & date_range
#ext_pps = ext & ref_pps & date_range

# exited the site prior to the 2 year rolling period but has an outcome lodged during the 2 year rolling period -> doesn't denominate - but doesn't numerate 
#date_before = df_pre['COMMENCEMENT_DATE'].loc[(df_pre['COMMENCEMENT_DATE'] < '2019-01-01')]

# doesn't denominate or numerate
#clm_date = df_pre['CLAIM_START_DATE'].loc[(df_pre['CLAIM_START_DATE'] >= '2019-01-01') & (df_pre['CLAIM_END_DATE'] <= '2021-01-08')]
#no_inc = date_before & clm_date # don't include in num/den

# ext_before = ext & date_before # don't include in num/den


# will count in denominator if sus_ea + more than 13 week + includes suspension period: 
# SUSPENSION IS BASED ON DATE, NOT CODE. Suspension date can occur in any phase
# 1) Suspension period exists: 
is_sus = (df_pre['SUSPENSION_DAYS'].isna() == False) # suspension dates exist and check commencement period 




df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Code', 'Site_Name',
       'ESA_Code', 'ESA Name', 'State', 'specialist_site_type_code',
       'COMMENCEMENT_DATE', 'EXIT_DATE', 'Claim_ID', 'Claim_Rate_Type',
       'Claim_Status', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE',
       'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE',
       'OUTCOME_TYPE', 'SUSPENSION_DAYS'],
      dtype='object')

In [12]:
df_pre['Claim_Rate_Type'].value_counts()

na                                                    269425
DES 4wk Outcome                                        56937
DES 13wk Full Outcome                                  29728
DES 26wk Full Outcome                                  20586
DES 52wk Full Outcome                                   7354
DES 26wk Full Outcome Education                         2284
DES 13wk Pathway Outcome                                2044
DES 13wk Full Outcome Education                         2043
DES 26wk Pathway Outcome                                 585
DES 13wk Pathway Outcome Education                       286
DES 52wk Pathway Outcome                                 268
DES 4wk Outcome Moderate Intellectual Disability P        35
DES 13wk Outcome Moderate Intellectual Disability         28
DES 26wk Outcome Moderate Intellectual Disability         16
DES 13wk Full Outcome Zero Dollar R4                       8
DES 52wk Outcome Moderate Intellectual Disability          6
Name: Claim_Rate_Type, d

In [13]:
find13 = df_pre["Claim_Rate_Type"].str.contains(r'DES 13wk Full Outcome')
find26 = df_pre["Claim_Rate_Type"].str.contains(r'DES 26wk Full Outcome')
find52 = df_pre["Claim_Rate_Type"].str.contains(r'DES 52wk Full Outcome')



# combine the placement_status_code and referral_phase_code :
df_pre['PLACE_REF'] = list(zip(df_pre.PLACEMENT_STATUS_CODE, df_pre.REFERRAL_PHASE_CODE))




### Filtering NaT for - Claim_available_date

- Validating the date period

In [14]:
df_pre = df_pre[df_pre['CLAIM_AVAILABLE_DATE'].isna() == False ]
df_pre['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_pre['CLAIM_AVAILABLE_DATE'])
df_pre['CLAIM_AVAILABLE_DATE'].min()

Timestamp('2001-01-01 00:00:00')

In [15]:

df_pre['CLAIM_AVAILABLE_DATE'].max()

Timestamp('2022-10-07 00:00:00')

### Cleaning Suspension weeks date
- Convert days -> weeks

In [16]:
df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Code', 'Site_Name',
       'ESA_Code', 'ESA Name', 'State', 'specialist_site_type_code',
       'COMMENCEMENT_DATE', 'EXIT_DATE', 'Claim_ID', 'Claim_Rate_Type',
       'Claim_Status', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE',
       'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE',
       'OUTCOME_TYPE', 'SUSPENSION_DAYS', 'PLACE_REF'],
      dtype='object')

## 13 Week Outcomes (Numerator + Denominator)
+ Creating a dictionary (key: Job_Seeker_ID, values: [all attributes for one row], [all attributes for second row], etc.) 
    - The values in the dictionary are the rows which correspond to the same Job_Seeker_ID -> but stored like so: 
    E.g. for job_seeker_id 1001:
    1001     :  [row 1]
                [row 2]
                [row 3]
    This allows for efficient access when identifying the smallest placement_seq_no for instance -> accessing row 1, rather than iterating through the dataframe. 
    - Can check index of the rows -> need to check whether if row 3 has a 26 week outcome, check if 13 week outcome is in row 2 or row 1. 
    - Check 13 week outcome by adding 13 weeks to commencement date by adding 13x7 days.

In [17]:
df_pre = df_pre.sort_values(by=['COMMENCEMENT_DATE'])
df_pre.head(10)

,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Code,Site_Name,ESA_Code,ESA Name,...,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS,PLACE_REF
186508,9386301003,NE,Pending,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Pending, NE)"
301873,9386301003,EA,Exited,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, EA)"
301874,9386301003,EA,Exited,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, EA)"
375254,9386301003,NE,Exited,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, NE)"
375252,9386301003,NE,Exited,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, NE)"
101732,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
101730,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
375251,9386301003,NE,Exited,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, NE)"
333192,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
333190,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AE71,AimBig Employment WOY WOY,4CEC,Central Coast NSW,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"


In [18]:
df_pre = df_pre.drop_duplicates()
df_pre = df_pre.sort_values(['COMMENCEMENT_DATE', 'CLAIM_START_DATE'])

g = df_pre.groupby(["Job_Seeker_ID"]) # remove commencement_date with NaT -> check if NaT values, filter these out and remove




- Groups -> one group containing entries that correspond to the same Job_Seeker_ID
srce: https://stackoverflow.com/questions/22691010/how-to-print-a-groupby-object


## 13 Week Numerator and Denominator Function - Pre-Quarterisation
- Get the lowest index value of the group (for each list)
- Create a new dataframe df_13: [Job_Seeker_ID, Num_13, Den_13] 
    - Contains the 13 week numerator and denominator for corresponding job seeker id
- Use pd.DateOffset instead of timedelta(days=....)
- Excludes in the denominator: participants who have achieved a 13 week pathway outcome

In [19]:
# list that contains the numerator codes: 
com_ea = com & ref_ea
com_pps = com & ref_pps
com_os = com & ref_os 
trn_pps = trn & ref_pps
trn_ea = trn & ref_ea 
num_codes = [('Commenced', 'EA'), ('Commenced', 'PPS'), ('Commenced', 'OS'), ('Transferred', 'PPS'), ('Transferred', 'EA')]


#EXT_EA: exited the site with assistance, or exited assistance
#EXT_PPS: exited the site with post placement support 
den_codes = [('Exited', 'EA'), ('Exited', 'PPS'), ('Transferred', 'PPS'), ('Transferred', 'EA')]

# participant has commenced at a site:
com_codes = [('Commenced', 'EA'), ('Commenced', 'PPS')]


Job_Seeker_ID is set as the index -> access by "g.index[0]"

In [20]:
com_codes


[('Commenced', 'EA'), ('Commenced', 'PPS')]

endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

In [21]:
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

In [22]:
df_pre['PLACEMENT_STATUS_CODE'].value_counts()

Commenced      58829
Pending        27097
Exited         22143
Suspended      13518
Transferred     4596
In Progress      545
Enquiry            7
Name: PLACEMENT_STATUS_CODE, dtype: int64

In [23]:
com_codes

[('Commenced', 'EA'), ('Commenced', 'PPS')]

In [24]:
def get_13_outcome(jobSeekerEntry, num_codes, den_codes, com_codes):
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry
    id = group.index.values.astype(int)[0] 
    # initialise numerator and denominator 
    num_13 = 0
    den_13 = 0
    df_13 = pd.DataFrame()
    # place id into dataframe
    df_13['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    week_13 = pd.DateOffset(days = 91)
    # Check number of entries in group
    targetStr = 'DES 13wk Full Outcome'
    num_entries = int(len(jobSeekerEntry))

    # CLAIM TYPES - 
    eduStr = ['DES 13wk Full Outcome Education', 'DES 13wk Pathway Outcome Education'] # denominate
    noClaim = 'na'
    pathwayStr = 'DES 13wk Pathway Outcome' # doesn't denominate, doesn't numerate
    valid_claims = ['Approved', 'Pending', 'Lodged'] #but excluding lodged, pending or approved FOR EDUCATION 13 week full outcomes
    exitDate = group.EXIT_DATE.values[0] # if exit date exists -> 

    # now loop ->
    j = 0
    comDate = group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_13 = comDate + week_13
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_13): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_13 = comDate + week_13 + sus_days
            else:
                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] == targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                
                # check if participant exits before claim_available_date ->
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_13):
                    
                    num_13 = 0 
                    den_13 = 0
                    
                else:
                    num_13 += 1
                    den_13 = 1
                    df_13['Num_13'] = [num_13]
                    df_13['Den_13'] = den_13
                    return df_13 

            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_13 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_13: # duration in assistance is less than 13 weeks
                    num_13 = 0 
                    den_13 = 0
                   
                else: # check duration in assistance is at least period_13 
                    num_13 = 0
                    den_13 = 1 #education outcomes are counted in the denominator
            elif group.Claim_Rate_Type.values[j] == pathwayStr and (group.Claim_Status.values[j] in valid_claims): # denominator excludes pathway outcomes
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_13):
                    num_13 = 0 
                    den_13 = 0
                else:
                    num_13 += 0 
                    den_13 = 1
            elif group.Claim_Rate_Type.values[j] == noClaim: 
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            
                if (exitDate < period_13):
                    num_13 = 0 
                    den_13 = 0
                else:
                    num_13 += 0 
                    den_13 = 1
            else: # either 26, 52 week claim
                num_13 = 0
                den_13 = 0
            
        
        # has not commenced ->
        else: 
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_13 = comDate + week_13
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_13): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_13 = comDate + week_13 + sus_days

            # check if code is SUS_EA: (Counted in denominator if completed more than 13 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_13:
                    if (group.Claim_Rate_Type.values[j] == targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_13 <= startBounds) or (period_13 >= endBounds):
                                num_13 = 0 
                                den_13 = 0 # out of bounds
                        else:
                            # then doesn't denominate: 
                            num_13 = 1
                            den_13 = 1

                      
                       
                    else:
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_13 <= startBounds) or (period_13 >= endBounds):
                            num_13 = 0 
                            den_13 = 0 # out of bounds
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 13wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_13 = 0 
                            else:
                                den_13 = 1
                       
                    
                else: # no claim available date:
                    if group.Claim_Rate_Type.values[j] == 'na':
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_13:
                            num_13 = 0
                            den_13 = 1

                    else:
                        num_13 = 0
                        den_13 = 0

            else: # TRN or EXT
                
                ext_tfn = ['Exited', 'Transferred']
                if group.PLACEMENT_STATUS_CODE.values[j] in ext_tfn:
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                    
                   

                    
                    # check claim rate types ->
                    if (group.Claim_Rate_Type.values[j] == targetStr) and (group.Claim_Status.values[j] in valid_claims): # valid claim
                       
                        if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                            
                            num_13 = 0
                            den_13 = 0
                        else:
                            num_13 += 1
                            den_13 = 1
                            df_13['Num_13'] = [num_13]
                            df_13['Den_13'] = den_13
                            return df_13  

                    elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_13 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                        if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                            num_13 = 0
                            den_13 = 0
                        else:
                            num_13 = 0
                            den_13 = 1


                    elif group.Claim_Rate_Type.values[j] == pathwayStr and (group.Claim_Status.values[j] in valid_claims): # denominator excludes pathway outcomes
                        if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                            num_13 = 0
                            den_13 = 0
                        else:
                            num_13 += 0 
                            den_13 = 1

                    elif group.Claim_Rate_Type.values[j] == noClaim: 
                        if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                            num_13 = 0
                            den_13 = 0
                        else:
                            num_13 += 0 
                            den_13 = 1

                    else: # either 26, 52 week claim
                        num_13 = 0
                        den_13 = 0
                else:
                    pass # pending
        
        j += 1

        
    df_13['Num_13'] = [num_13]
    df_13['Den_13'] = den_13
    return df_13
 




        
### Continuous 
jobEntry = g.get_group(18614400)
get_13_outcome(jobEntry, num_codes, den_codes, com_codes)

,Job_Seeker_ID,Num_13,Den_13
0,18614400,1,1


In [25]:
### Continuous 
jobEntry = g.get_group(9264871709)
get_13_outcome(jobEntry, num_codes, den_codes, com_codes)

,Job_Seeker_ID,Num_13,Den_13
0,9264871709,0,1


In [26]:
jobEntry.Claim_Rate_Type.value_counts()

na    10
Name: Claim_Rate_Type, dtype: int64

In [27]:


dups_color = df_pre.pivot_table(columns=['Job_Seeker_ID'], aggfunc='size')
print (dups_color)
# need to drop duplicates such that Job_Seeker_ID is unique: 


Job_Seeker_ID
44710         20
61780         12
70560         20
82960         32
85140         48
              ..
9983739909    16
9985676409    48
9986691809    36
9996900006    80
9999230109    10
Length: 4976, dtype: int64


### Aggregate 13_week_outcome function

Looping through df_comb and passing each group to the get_13_outcome function

In [28]:
# Number of distinct Job_Seeker_IDs from df_pre:
distinct = df_pre['Job_Seeker_ID'].unique()
distinct

array([9386301003, 8653691003, 7870290004, ..., 8771045004,  615882709,
        690092007], dtype=int64)

In [29]:
# adds outcome to df_13 ->

def agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes):
    df_13 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_13', 'Den_13']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome

    ids = df_pre['Job_Seeker_ID'].unique()
    for id in ids:
        if math.isnan(id) or id == 0:
            pass
        else:
            df_entry = g.get_group(id)
            f = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
            df_13 = df_13.append(f)
            

    return df_13 # returns output from get_13_outcome 

# map outcomes to dataframe -> one pass

# testing hurstville
agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)

,Job_Seeker_ID,Num_13,Den_13
0,9386301003,0,1
0,8653691003,0,1
0,7870290004,0,1
0,6498977509,0,0
0,6638331004,0,0
...,...,...,...
0,5766520,1,1
0,2836891309,1,1
0,8771045004,0,1
0,615882709,1,1


### Testing sites -> job seekers - AUBURN

In [30]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')


x = agg_outcomes_13(jobSeekers, g, num_codes, den_codes, com_codes)
x['Den_13'].sum()

4

In [31]:
x = agg_outcomes_13(jobSeekers, g, num_codes, den_codes, com_codes)
x['Num_13'].sum()



1

### separating the sub_types from the performance measure 


In [32]:
df_13_new = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_13_full', 'Den_13_full'])

for index, row in df_13_new.iterrows():
    
    g_key = row['Job_Seeker_ID']
    g_group = g_supp.get_group(g_key)
    g_claims = g_group.CLAIM_RATE_TYPE.values
    if 'DES 13wk Full Outcome' in g_claims:
        #   get the relevant dataframe matches
            row_match = df_13[df_13['Job_Seeker_ID'] == g_key]
            Num_13_old = row_match['Num_13']
            Den_13_old = row_match['Den_13']
            data = pd.DataFrame({'Job_Seeker_ID' : [g_key], 'Num_13_full': Num_13_old, 'Den_13_full' : Den_13_old})
            df_13_new = df_13_new.append(data) 





### Pre-Quarterisation Output File:

In [33]:
### Testing the aggregate function for 13 outcomes
df_13 = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)

print('Number of IDs for 13 week outcomes: ' + str(len(df_13)))
df_13

Number of IDs for 13 week outcomes: 4976


,Job_Seeker_ID,Num_13,Den_13
0,9386301003,0,1
0,8653691003,0,1
0,7870290004,0,1
0,6498977509,0,0
0,6638331004,0,0
...,...,...,...
0,5766520,1,1
0,2836891309,1,1
0,8771045004,0,1
0,615882709,1,1


In [34]:
# output file
df_13.to_csv('df_13_pre.csv')


### Post-Quarterisation for 13 Week Outcomes - Numerator doesn't change, only the DENOMINATOR
#### Check - The numerator achieved date -> 13 weeks after commencement date 
- Denominate each quarter if they do not numerate. For each quarter a participant is eligible to record an outcome, i.e. been in assistance at least 13 weeks - if no outcome is recorded, then the participant continues denominating. 
- If the quarter falls outside the 2 year period - record is not included
- not included if time to achieve an outcome < 13 weeks
- Need to break up the 2 year rolling period into quarters -> 8 quarters (Based on business year)

endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')

startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

1. Given the 2 year rolling period - breaking the period up to quarters
Format: nested list [[start_date, end_date],...]
- 8 quarters


In [35]:
q_start_dates = pd.date_range('2020-10-01', '2022-10-07', freq='BQS')
q_end_dates = pd.date_range('2020-10-01', '2022-10-07', freq='BQ-DEC')
print("All the business quarterly begin dates of 2020-22:")
print(q_start_dates.values)
print("\nAll the business quarterly end dates of 2020-22:")
print(q_end_dates.values)

print("\n")

All the business quarterly begin dates of 2020-22:
['2020-10-01T00:00:00.000000000' '2021-01-01T00:00:00.000000000'
 '2021-04-01T00:00:00.000000000' '2021-07-01T00:00:00.000000000'
 '2021-10-01T00:00:00.000000000' '2022-01-03T00:00:00.000000000'
 '2022-04-01T00:00:00.000000000' '2022-07-01T00:00:00.000000000'
 '2022-10-03T00:00:00.000000000']

All the business quarterly end dates of 2020-22:
['2020-12-31T00:00:00.000000000' '2021-03-31T00:00:00.000000000'
 '2021-06-30T00:00:00.000000000' '2021-09-30T00:00:00.000000000'
 '2021-12-31T00:00:00.000000000' '2022-03-31T00:00:00.000000000'
 '2022-06-30T00:00:00.000000000' '2022-09-30T00:00:00.000000000']




Quarters:
Q1: '2020-10-01', '2020-12-31'
Q2: '2021-01-01', '2021-03-31'
Q3: '2021-04-01', '2021-06-30'
Q4: '2021-07-01', '2021-09-30'
Q5: '2021-10-01', '2021-12-31'
Q6: '2022-01-03', '2022-03-31'
Q7: '2022-04-01', '2022-06-30'
Q8: '2022-07-01', '2022-09-30'

In [36]:
id = df_13.Job_Seeker_ID.values[6]
i = 0
n = len(g.get_group(id))
group = g.get_group(id)
group.Claim_Rate_Type.values

if 'DES 13wk Full Outcome' in group.Claim_Rate_Type.values:
    print(group.Claim_Rate_Type.values)

outcomeInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
outcomeInd


# while i < n:
    # out = group.Claim_Rate_Type.values[i]
    # print(out)
    # print(dt)
    # i += 1

0

Quarters:
Q1: '2020-10-01', '2020-12-31'
Q2: '2021-01-01', '2021-03-31'
Q3: '2021-04-01', '2021-06-30'
Q4: '2021-07-01', '2021-09-30'
Q5: '2021-10-01', '2021-12-31'
Q6: '2022-01-03', '2022-03-31'
Q7: '2022-04-01', '2022-06-30'
Q8: '2022-07-01', '2022-09-30'

In [37]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]
z = 0
s = 0
e = 1
while z < len(qtrs):
    start = datetime.strptime(qtrs[z][s], '%Y-%m-%d')
    qtrs[z][s] = start
    end = datetime.strptime(qtrs[z][e], '%Y-%m-%d')
    qtrs[z][e] = end
    
    z += 1

### Edge Cases for Post Quarterisation:
- If participant does not record a 13 week outcome ever -> denominates the max 8 times.
- If time to achieve an outcome is less than 13 weeks -> don't denominate. -> but this should be in the df_13 already 
- If quarter falls outside of the two year assessment period, then not included.


In [38]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]


qNo = 0
while qNo < len(qtrs):
    s = (qtrs[qNo][0], "%Y-%m-%d") # start date of quarter
    e = (qtrs[qNo][1], "%Y-%m-%d")  # end date of quarter
    qNo += 1


i = 0
id = df_13.Job_Seeker_ID.values
Den_13 = 0 
Num_13 = 0
while i < len(id):
    id2 = id[i]
    
    x = 0
    group = g.get_group(id2)
    z = 0
    while z < len(group.CLAIM_AVAILABLE_DATE.values):
        
        claim = group.Claim_Rate_Type.values[z] 
        if claim == 'DES 13wk Full Outcome':  # first check for the type of claim rate -> find 13 week outcome

         
            Num_13 = 1
            # final denominator -> assign to current count of denominator -> ask if they denominate during the numerating
            Den_13 += 1 
            break
       
       
        else:
            Den_13 += 1
            pass 
            # just count through the denominators when the claim exists and is recorded
        z += 1
        
    i += 1


In [39]:
df_pre.dtypes

Job_Seeker_ID                          int64
REFERRAL_PHASE_CODE                   object
PLACEMENT_STATUS_CODE                 object
PLACEMENT_TYPE                        object
Program                               object
Contract_ID                           object
Site_Code                             object
Site_Name                             object
ESA_Code                              object
ESA Name                              object
State                                 object
specialist_site_type_code             object
COMMENCEMENT_DATE             datetime64[ns]
EXIT_DATE                             object
Claim_ID                               int64
Claim_Rate_Type                       object
Claim_Status                          object
CLAIM_START_DATE              datetime64[ns]
CLAIM_AVAILABLE_DATE          datetime64[ns]
CLAIM_END_DATE                datetime64[ns]
SUSPENSION_FROM_DATE          datetime64[ns]
SUSPENSION_ACTUAL_END_DATE    datetime64[ns]
OUTCOME_TY

In [40]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]


endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')

startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

def post_quarter_13(df_13, df_pre, g, qtrs):
    '''Post quarterisation performance for 13 week full outcomes. Receives df_pre as an input, and outputs the post_quarterised denominators for each eligible participant.
    df_13 is used to check if a participant has numerated, df_pre is used to feed in the job_seeker_id to 'g' - the groupBy object and check whether the job seeker denominates - if no numerator in the df_13 exists.'''
    # maximum amount of times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_13', 'Den_13']) # Dataframe template for result
    
    claimExists = False
    
    i = 0
    while i < len(df_13):
        js_id = df_13.Job_Seeker_ID.values[i]
        num_13 = df_13.Num_13.values[i] 
        den_13 = df_13.Den_13.values[i]
        

        
        if (num_13 == 0) and (num_13 == den_13): # if a user has recorded 13_week_outcome in previous 2 year rolling pd:
           
            Num_13 = 0
            Den_13 = 0

        
        elif num_13 == 1: # participant has numerated ->
            Num_13 = 1
            Den_13 = 1


        else: # numerator is 0 -> check if participant denominates with each quarter
            Num_13 = 0
            Den_13 = 0
            x = 0
            group = g.get_group(js_id)
            n = len(group)
            week_13 = pd.DateOffset(days = 91)
            while x < n:
                comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0]) # commencement date should be the first index 
                claim = group.Claim_Rate_Type.values[x]
                availDate = group.CLAIM_AVAILABLE_DATE.values[x]
                exitDate = pd.to_datetime(group.EXIT_DATE.values[x])

                if num_13 == maxNum or den_13 == maxDen:
                    pass # return -> denominated maxDen = 8 times, or maxNum/maxDen
                else: 
                    # looping through the outcomes
                    if (group.PLACEMENT_STATUS_CODE.values[x] == 'Exited') and (group.SUSPENSION_DAYS.values[x] > 0): # current entry is showing Exited -> check if exitDate is  
                        period_13 = comDate + week_13 
                        if (group.SUSPENSION_FROM_DATE.values[x] >= comDate) and (group.SUSPENSION_ACTUAL_END_DATE.values[x] <= period_13): 
                            # then suspension is applicable to this outcome and period
                            period_13 = comDate + week_13 + pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[x]))
                            # check if participant has exited before the 13 week outcome was due: 
                            if exitDate < period_13: 
                                # then don't include at all -> wipe all numerator/denominator
                                Num_13 = 0 
                                Den_13 = 0
                                pass
                        
                        

                    # loop through the quarters and compare with claim_available_date -> check which quarter it occurs in
                    elif (claim == 'DES 13wk Pathway Outcome') or (claim == 'DES 13wk Full Outcome'):
                        if (claim == 'DES 13wk Full Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 13 week outcome claim exists - validate the number for the denominator -> reset
                            Den_13 = 1 
                            claimExists = True 
                            Num_13 = 1
                            #get the availDate -> check which quarter it falls under
                        elif (claim == 'DES 13wk Pathway Outcome'):
                            Den_13 = 0 
                            Num_13 = 0
                            pass

                        # check if a 13 week full outcome exists in ANY of the group values
                            
                        z = 0
                        s = 0
                        e = 1
                        while z < len(qtrs):
                            start = pd.to_datetime(qtrs[z][s])
                            end = pd.to_datetime(qtrs[z][e])
                            if Den_13 == maxDen or Num_13 == maxNum:
                                break
                            else:
                                if 'DES 13wk Full Outcome' in group.Claim_Rate_Type.values:
                                    # if the iteration -> loop is at the beginning of the current group -> get the commencement date of the FIRST row for the 13 week full outcome: 
                                    outcomeInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                                    comDate = group.COMMENCEMENT_DATE.values[outcomeInd]
                                    # get EXIT date for this here
                                    exitDate = pd.to_datetime(group.EXIT_DATE.values[outcomeInd]) 

                                    ## TO-DO - investigate whether a SUSPENSION period occurs here and if exitDate <= period_13 (comDate + suspension_period + 13 weeks)
                                elif ('DES 13wk Full Outcome' not in group.Claim_Rate_Type.values ) and ('DES 13wk Pathway Outcome' in group.Claim_Rate_Type.values):    # all values are 13 week PATHWAY outcomes
                                    pathwayInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                                    comDate = group.COMMENCEMENT_DATE.values[pathwayInd]
                                    exitDate = pd.to_datetime(group.EXIT_DATE.values[pathwayInd])

                                # check here how many times the participant denominates ->
                                if (comDate >= start) and (comDate <= end) and (group.PLACEMENT_STATUS_CODE.values[x] == 'Commenced'): # check if the commencement date is in the current quarter: 
                                    period_13 = comDate + week_13 # denominates if and only if at least 13 weeks has passed: 13 weeks + comDate -> falls into the current quarter, if not pass

                                    if (exitDate >= start) and (exitDate <= end) and (period_13 >= start) and (period_13 <= end): #exit date is in the current quarter -> then denominates and then exists
                                        Den_13 += 1 
                                        break
                                    elif (availDate >= start) and (availDate <= end) and (period_13 >= start) and (period_13 <= end): # in the current quarter -> exit date not counted
                                        Den_13 += 1

                                    

                                else: # commencement date has already passed 
                                    # need to stop at participant EXIT_DATE - get exit date of the 13 week full outcome or pathway outcome:
                                    exitDate = pd.to_datetime(group.EXIT_DATE.values[0]) # exit date should be recorded FOR ALL
                                    period_13 =  comDate + week_13
                                    # check if exitDate is within the current quarter being traversed through
                                    if (exitDate >= start) and (exitDate <= end) and (period_13 >= start) and (period_13 <= end): #exit date is in the current quarter -> then denominates and then exists
                                        Den_13 += 1 
                                        break
                                    else:
                                        Den_13 += 1 #continues denominating for each quarter until claim is recorded
                            
                        
                            z += 1
                        
                    elif claim == 'DES 52wk Full Outcome' or claim == 'DES 26wk Full Outcome':
                        Num_13 = 0 
                        Den_13 = 0 
                        # 13 week outcome may already be recorded -> or may be in later entries
                        pass 
                    # after the IF statement -> if claimExists == True, and denominator has been counted until the claim available date -> then return the denominated example -> break
                    # else -> if claimExists == False, denominates max 8 times


                    else: 
                        # denominates
                        Den_13 += 1

                x += 1

        if Den_13 >= maxDen:
            Den_13 = 8  # set threshold
        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_13' : Num_13, 'Den_13' : Den_13})
        df_post = df_post.append(data)


            

            
        
        i += 1
            





    return df_post # returns post-quarterisation dataframe 


test = post_quarter_13(df_13, df_pre, g, qtrs)
test.head(10)

,Job_Seeker_ID,Num_13,Den_13
0,9386301003,0,8
0,8653691003,0,8
0,7870290004,0,8
0,6498977509,0,0
0,6638331004,0,0
0,2792458409,1,1
0,8538100019,0,2
0,9277440019,0,2
0,2580218019,0,0
0,687762409,0,0


In [41]:
df_13_post = test.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})
df_13_post

,Job_Seeker_ID,Num_13_full,Den_13_full
0,9386301003,0,8
0,8653691003,0,8
0,7870290004,0,8
0,6498977509,0,0
0,6638331004,0,0
...,...,...,...
0,5766520,1,1
0,2836891309,1,1
0,8771045004,0,8
0,615882709,1,1


In [42]:
df_13_post.to_csv('C:/Users/AManalo/star_ratings_new/df_13_post.csv')

# 26 Week Outcomes

### Eligible 26 week outcomes:
Included:
- Full 26 week outcomes (excluding Education - only vacancy )
- Pathway 26 week outcomes (worth 5%)
- Bonus/work assist outcomes 

Excluded:
- Zero dollar outcomes 
- Education outcomes

1) Renaming Num_26 and Den_26 => Num_path_26, Den_path_26, Num_wrkast_26, den_wrkast_26 (work assist/bonus), Num_26 -> Num_full_26, Den_full_26


### 1) Pre-Quarterisation

Difference to the existing pre-quarterisation function: 

    1.  Get the date from the [13 week outcome] claim_available_date - in which the 13 week outcome was achieved (not commencement_date like with 13 week outcome)
        done

    NOW: assuming that the data is correct - continue the pre-quarterisation function 


    2. Add 13 weeks to the date from (1) excluding suspension periods. Check if EXIT_DATE occurs before 26 week is recorded -> excluded. 

    3. If there exists a 26 week eligible outcome within the 2 year rolling period (after the 13_week_claim_available_date) –> NUM_26 = 1, DEN_26 = 1

    4. Otherwise: NUM_26 = 0, DEN_26 = 1

    5. If 26 week outcome ELIGIBILITY DATE (26 weeks from 13 week outcome) falls outside the assessment period, then -> NUM_26 = 0, DEN_26 = 0 

Confirm types of eligible 26 week outcomes to be taken <- from Chiara’s email. 





### Importing supp_apv_stream_data (extension of assessment period)
- extends assessment period by 6 months prior to the start date
- only checked if in the current assessment period, there exists 26 or 52 week outcome and NO prior outcome to compare with. 
- pre-process the file - to ensure data is of the correct format

In [43]:
df_supp = pd.read_csv("C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/supp_apv_stream_data.csv") 
# rename columns to capital:
df_supp.columns = map(str.upper, df_supp.columns)
df_supp.dtypes

COMMENCEMENT_DATE                    object
PROGRAM_TYPE_CODE                    object
JR_JOBSEEKER_ID                       int64
JOB_SEEKER_ID                         int64
PLAC_JOBSEEKER_ID                     int64
PLACEMENT_TYPE                       object
PROGRAM                              object
CONTRACT_ID                          object
SITE_CODE                            object
SITE_NAME                            object
ESA_CODE                             object
ESA NAME                             object
STATE                                object
SPECIALIST_SITE_TYPE_CODE            object
REFERRAL_DATE                        object
REFERRAL_PHASE_CODE                  object
PLACEMENT_STATUS_CODE                object
PHASE_CHANGED                       float64
STATUS_CHANGED                        int64
PHASE_COMMENCEMENT_DATE              object
TIME_IN_PROGRAM_WEEKS               float64
PERIOD_OF_SERVICE_WEEKS             float64
DISABILITY_TYPE                 

--------------

### Pre-processing df_supp
1. Cleaning columns
2. Creating another groupBy object -> this is checked when the previous outcome (prior to the current 26/52 outcome) is missing in the 2Y period. 


In [44]:


# if both are NA -> then no anchor date



df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna()) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna()), None, None)


# PPS_START_DATE doesnt exist but overriden_outcome_started_on exists 
df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna()) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna() == False), df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'], None)
# overriden_outcome_started_on doesn't exists, but PPS_START_DATE existd
df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna() == False) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna()), df_supp['PPS_START_DATE'], df_supp['COMMENCEMENT_DATE_NEW'])

# if both exist -> take overriden_outcome_started_on
df_supp['COMMENCEMENT_DATE_NEW'] = np.where((df_supp['PPS_START_DATE'].isna() == False) & (df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'].isna() == False), df_supp['OVERRIDDEN_OUTCOME_STARTED_ON'], df_supp['COMMENCEMENT_DATE_NEW'])

# drop rows where COMMENCEMENT_DATE_NEW is NULL -> no valid placements
df_supp = df_supp[df_supp['COMMENCEMENT_DATE_NEW'].isna() == False]
# drop None
df_supp = df_supp[df_supp['COMMENCEMENT_DATE_NEW'] != None]

# dropping the original "COMMENCEMENT_DATE" ->
df_supp = df_supp.drop(columns=['COMMENCEMENT_DATE'])

df_supp = df_supp.rename(columns={'COMMENCEMENT_DATE_NEW' : 'COMMENCEMENT_DATE'})

# subset: 
df_supp = df_supp[["JOB_SEEKER_ID", 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE', 'PLACEMENT_TYPE', 'COMMENCEMENT_DATE', 'EXIT_DATE', 'CLAIM_ID', 'CLAIM_RATE_TYPE', 'CLAIM_STATUS', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE']]


df_supp = df_supp.sort_values(by=['JOB_SEEKER_ID', 'CLAIM_START_DATE'])

# converting data types + converting NaT to out of range dates (i.e. 2001)
df_supp['JOB_SEEKER_ID'] = pd.to_numeric(df_supp['JOB_SEEKER_ID'], errors='coerce')
df_supp = df_supp[df_supp['JOB_SEEKER_ID'].isna() != True]
df_supp['JOB_SEEKER_ID'] = df_supp['JOB_SEEKER_ID'].astype('int64')



# need to convert column into string first -> then back to date
dict_types = {'SUSPENSION_FROM_DATE' : 'str', 'COMMENCEMENT_DATE' : 'str', 'CLAIM_START_DATE' : 'str', 'CLAIM_AVAILABLE_DATE' : 'str', 'CLAIM_END_DATE' : 'str', 'SUSPENSION_ACTUAL_END_DATE' : 'str'}
df_supp = df_supp.astype(dict_types)
fillVal = pd.to_datetime('2001-12-23', format='%Y-%m-%d')



df_supp['SUSPENSION_FROM_DATE'] = pd.to_datetime(df_supp['SUSPENSION_FROM_DATE'], format='%Y-%m-%d')
df_supp['SUSPENSION_FROM_DATE'] = df_supp['SUSPENSION_FROM_DATE'].fillna(fillVal)

df_supp['SUSPENSION_ACTUAL_END_DATE'] = pd.to_datetime(df_supp['SUSPENSION_ACTUAL_END_DATE'], format='%Y-%m-%d')
df_supp['SUSPENSION_ACTUAL_END_DATE'] = df_supp['SUSPENSION_ACTUAL_END_DATE'].fillna(fillVal)

df_supp['COMMENCEMENT_DATE'] = pd.to_datetime(df_supp['COMMENCEMENT_DATE'], format='%Y-%m-%d')
df_supp['COMMENCEMENT_DATE'] = df_supp['COMMENCEMENT_DATE'].fillna(fillVal)


df_supp['CLAIM_START_DATE'] = pd.to_datetime(df_supp['CLAIM_START_DATE'], format='%Y-%m-%d')
df_supp['CLAIM_START_DATE'] = df_supp['CLAIM_START_DATE'].fillna(fillVal)

df_supp['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_supp['CLAIM_AVAILABLE_DATE'], format='%Y-%m-%d')
df_supp['CLAIM_AVAILABLE_DATE'] = df_supp['CLAIM_AVAILABLE_DATE'].fillna(fillVal)

df_supp['CLAIM_END_DATE'] = pd.to_datetime(df_supp['CLAIM_END_DATE'], format='%Y-%m-%d', errors='coerce')
df_supp['CLAIM_END_DATE'] = df_supp['CLAIM_END_DATE'].fillna(fillVal)



endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')


df_supp['COMMENCEMENT_DATE'] = df_supp["COMMENCEMENT_DATE"].loc[(df_supp['COMMENCEMENT_DATE'] <= endDate )]

df_supp["CLAIM_RATE_TYPE"] = df_supp["CLAIM_RATE_TYPE"].replace(np.nan, '', regex=True)

df_supp["CLAIM_RATE_TYPE"] = df_supp["CLAIM_RATE_TYPE"].astype('str')

# remove education outcome_type, or NaN outcome_type
df_supp = df_supp.loc[:,~df_supp.columns.duplicated()].copy()

# getting the period between suspension_from_date and suspension_actual_end_date in weeks
df_supp['SUSPENSION_DAYS'] = (df_supp['SUSPENSION_ACTUAL_END_DATE'] - df_supp['SUSPENSION_FROM_DATE']).apply(lambda x: x/np.timedelta64(1, 'D'))



df_supp['CLAIM_RATE_TYPE'].value_counts()



                                                      269425
DES 4wk Outcome                                        56937
DES 13wk Full Outcome                                  31641
DES 26wk Full Outcome                                  21626
DES 52wk Full Outcome                                   7615
DES 13wk Full Outcome Education                         2795
DES 26wk Full Outcome Education                         2623
DES 13wk Pathway Outcome                                2068
DES 26wk Pathway Outcome                                 597
DES 13wk Pathway Outcome Education                       500
DES 52wk Pathway Outcome                                 283
DES 4wk Outcome Moderate Intellectual Disability P        35
DES 13wk Outcome Moderate Intellectual Disability         28
DES 26wk Outcome Moderate Intellectual Disability         16
DES 13wk Full Outcome Zero Dollar R4                       8
DES 52wk Outcome Moderate Intellectual Disability          6
Name: CLAIM_RATE_TYPE, d

In [45]:
# combine the placement_status_code and referral_phase_code :
df_supp['PLACE_REF'] = list(zip(df_supp.PLACEMENT_STATUS_CODE, df_supp.REFERRAL_PHASE_CODE))

df_supp = df_supp[df_supp['CLAIM_AVAILABLE_DATE'].isna() == False ]
df_supp['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_supp['CLAIM_AVAILABLE_DATE'])
df_supp['CLAIM_AVAILABLE_DATE'].min()


Timestamp('2001-12-23 00:00:00')

In [46]:
df_supp['CLAIM_AVAILABLE_DATE'].max()

Timestamp('2022-10-07 00:00:00')

In [47]:
df_supp['SUSPENSION_DAYS'] = np.where(df_supp['SUSPENSION_DAYS'].isna(), 0, df_supp['SUSPENSION_DAYS'])



### groupBy object -> "g_supp"

In [48]:
df_supp = df_supp.sort_values(['COMMENCEMENT_DATE', 'CLAIM_START_DATE'])
# drop duplicates:

g_supp = df_supp.groupby(["JOB_SEEKER_ID"]) # remove commencement_date with NaT -> check if NaT values, filter these out and remove


### Pre-quarterisation function (26 week outcome)

In [49]:
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

In [50]:
df_entry = g.get_group(451780)
df_entry

,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Code,Site_Name,ESA_Code,ESA Name,...,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS,PLACE_REF
17453,451780,OS,Exited,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Exited, OS)"
32499,451780,PPS,Commenced,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, PPS)"
361279,451780,OS,Exited,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Exited, OS)"
397625,451780,PPS,Commenced,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, PPS)"
402503,451780,OS,Commenced,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2023-02-14,2023-02-27,NaN,13,"(Commenced, OS)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443800,451780,EA,Suspended,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,DES 52wk Pathway Outcome,Approved,2022-01-24,2022-01-24,2022-07-31,2023-02-14,2023-02-27,Employment,13,"(Suspended, EA)"
427089,451780,OS,Exited,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,DES 52wk Pathway Outcome,Approved,2022-01-24,2022-01-24,2022-07-31,2023-02-14,2023-02-27,Employment,13,"(Exited, OS)"
402502,451780,OS,Commenced,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,DES 52wk Pathway Outcome,Approved,2022-01-24,2022-01-24,2022-07-31,2023-02-14,2023-02-27,Employment,13,"(Commenced, OS)"
397623,451780,PPS,Commenced,Vacancy Placement,Dms3,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,...,DES 52wk Pathway Outcome,Approved,2022-01-24,2022-01-24,2022-07-31,2023-02-14,2023-02-27,Employment,13,"(Commenced, PPS)"


In [51]:
def get_26_outcome(jobSeekerEntry, num_codes, den_codes, com_codes): # takes a row (groupBy on Job_Seeker_ID, list of eligible numerator phase codes + denominator phase codes
    # sort jobSeekerEntry by COMMENCEMENT_DATE
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry # group containing the attributes for one job_seeker_ID
    id = group.index.values.astype(int)[0]
    num_26_full = 0
    den_26_full = 0
    num_26_path = 0 
    den_26_path = 0
    num_26_wrkast = 0 
    den_26_wrkast = 0
    df_26 = pd.DataFrame()

    # place id into dataframe 
    df_26['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    # initialise the variables in df_26
    df_26['Num_26_full'] = [num_26_full]
    df_26['Den_26_full'] = 0
    df_26['Num_26_path'] = [num_26_path]
    df_26['Den_26_path'] = 0
    df_26['Num_26_wrkast'] = [num_26_wrkast]
    df_26['Den_26_wrkast'] = 0

    week_13 = pd.DateOffset(days = 91)
    week_26 = pd.DateOffset(days = 182) # 26 weeks -> because adding from commencement_date -> but maybe it should be from 13 week ?
    eduStr = ['DES 26wk Full Outcome Education']
    # Check number of entries in group
    targetStr = ['DES 26wk Full Bonus', 'DES 26wk Full Outcome', 'DES 26wk Pathway Outcome', 'DES 26wk Work Assist Outcome'] #valid outcomes
    num_entries = int(len(jobSeekerEntry))
    noClaim = 'na'

    valid_claims = ['Approved', 'Pending', 'Lodged'] 
    exitDate = group.EXIT_DATE.values[0] # if exit date exists ->  exit date is the same for all entries

    # now loop ->
    j = 0
    comDate =  group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_26 = comDate + week_26
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                # check suspension days is in the 2 year assessment period:
                 if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_26 = comDate + week_26 + sus_days
            else:
                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                # check if participant exits before claim_available_date ->
            
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_26):
                        num_26_full = 0
                        den_26_full = 0
                        num_26_path = 0 
                        den_26_path = 0
                        num_26_wrkast = 0 
                        den_26_wrkast = 0

                else: # check claim type 
                    if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                        num_26_full = 1
                        den_26_full = 1
                        df_26['Num_26_full'] = [num_26_full]
                        df_26['Den_26_full'] = 1
                        return df_26
                    elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                        num_26_wrkast = 1
                        den_26_wrkast = 1
                        df_26['Num_26_wrkast'] = [num_26_wrkast]
                        df_26['Den_26_wrkast'] = 1
                        return df_26
                    elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                        num_26_path = 1
                        den_26_path = 1
                        df_26['Num_26_path'] = [num_26_path]
                        df_26['Den_26_path'] = 1
                        return df_26
            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_26: # duration in assistance is less than 26 weeks
                    num_26_full = 0
                    den_26_full = 0
                    num_26_path = 0 
                    den_26_path = 0
                    num_26_wrkast = 0 
                    den_26_wrkast = 0
                   
                else: # check duration in assistance is at least period_26
                    num_26_full = 0
                    den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

            elif group.Claim_Rate_Type.values[j] == noClaim: 
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_26: # duration in assistance is less than 26 weeks
                    num_26_full = 0
                    den_26_full = 0
                    num_26_path = 0 
                    den_26_path = 0
                    num_26_wrkast = 0 
                    den_26_wrkast = 0
                else: # just denominate in the full outcome
                    num_26_full = 0 
                    den_26_full = 1
            else: # either 14 or 52 week claim
                num_26_full = 0
                den_26_full = 0

        else: # has not commenced ->
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_26 = comDate + week_26
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_26 = comDate + week_26 + sus_days
                
            # check if code is SUS_EA: (Counted in denominator if completed more than 26 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_26:
                    if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_26 <= startBounds) or (period_26 >= endBounds):
                                num_26_full = 0 
                                den_26_full = 0 # out of bounds
                        else:
                            # Check the claim type ->
                            if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                num_26_full = 1
                                den_26_full = 1
                                df_26['Num_26_full'] = [num_26_full]
                                df_26['Den_26_full'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                num_26_wrkast = 1
                                den_26_wrkast = 1
                                df_26['Num_26_wrkast'] = [num_26_wrkast]
                                df_26['Den_26_wrkast'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                num_26_path = 1
                                den_26_path = 1
                                df_26['Num_26_path'] = [num_26_path]
                                df_26['Den_26_path'] = 1
                                return df_26
                    else:
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_26 <= startBounds) or (period_26 >= endBounds):
                            num_26_full = 0 
                            den_26_full = 0 # out of bounds
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 26wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_26_full = 0 
                                den_26_path = 1
                            else: # check the claim types and denominate these
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # full outcome
                                    den_26_full = 1
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]:
                                    den_26_wrkast = 1
                                else: # NA claim
                                    den_26_full = 1

                else: # no claim_availble date or claim available date is less than period_26
                    if group.CLAIM_AVAILABLE_DATE.values[j] < period_26 and group.Claim_Rate_Type.values[j] == 'na':
                        # check duration in assistance
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_26: # exits after 26 weeks
                            num_26_full = 0
                            den_26_full = 1
                        else: # exits before 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                    else: # could be suspended but still record a claim ->
                        comDate = group.COMMENCEMENT_DATE.values[j]
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        period_26 = comDate + week_26
                        # check suspensions
                        if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                            # check suspension days is in the 2 year assessment period:
                            if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                                sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                                period_26 = comDate + week_26 + sus_days
                        else:
                            pass
 
                       
                        # claim_available_date could be valid and occur before period_26:
                        # claim rate types:
                        if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                            # check if participant exits before claim_available_date ->
                        
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if (exitDate < period_26):
                                    num_26_full = 0
                                    den_26_full = 0
                                    num_26_path = 0 
                                    den_26_path = 0
                                    num_26_wrkast = 0 
                                    den_26_wrkast = 0

                            else: # check claim type 
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                    num_26_full = 1
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                    num_26_wrkast = 1
                                    den_26_wrkast = 1
                                    df_26['Num_26_wrkast'] = [num_26_wrkast]
                                    df_26['Den_26_wrkast'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                    num_26_path = 1
                                    den_26_path = 1
                                    df_26['Num_26_path'] = [num_26_path]
                                    df_26['Den_26_path'] = 1
                                    return df_26
                        elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_26: # duration in assistance is less than 26 weeks
                                num_26_full = 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0
                            
                            else: # check duration in assistance is at least period_26
                                num_26_full = 0
                                den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

                        elif group.Claim_Rate_Type.values[j] == noClaim: 
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_26: # duration in assistance is less than 26 weeks
                                num_26_full = 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0
                            else: # just denominate in the full outcome
                                num_26_full = 0 
                                den_26_full = 1
                        else: # either 14 or 52 week claim
                            num_26_full = 0
                            den_26_full = 0
                
                                        
            else: # TRN or EXT
                ext_tfn = ['Exited', 'Transferred']
                if group.PLACEMENT_STATUS_CODE.values[j] in ext_tfn:
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[j])

                    if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                        # check if participant exits before claim_available_date ->
                    
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if (exitDate < period_26):
                                num_26_full = 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0

                        else: # check claim type 
                            if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                num_26_full = 1
                                den_26_full = 1
                                df_26['Num_26_full'] = [num_26_full]
                                df_26['Den_26_full'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                num_26_wrkast = 1
                                den_26_wrkast = 1
                                df_26['Num_26_wrkast'] = [num_26_wrkast]
                                df_26['Den_26_wrkast'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                num_26_path = 1
                                den_26_path = 1
                                df_26['Num_26_path'] = [num_26_path]
                                df_26['Den_26_path'] = 1
                                return df_26
                    elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_26: # duration in assistance is less than 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                            num_26_path = 0 
                            den_26_path = 0
                            num_26_wrkast = 0 
                            den_26_wrkast = 0
                        
                        else: # check duration in assistance is at least period_26
                            num_26_full = 0
                            den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

                    elif group.Claim_Rate_Type.values[j] == noClaim: 
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_26: # duration in assistance is less than 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                            num_26_path = 0 
                            den_26_path = 0
                            num_26_wrkast = 0 
                            den_26_wrkast = 0
                        else: # just denominate in the full outcome
                            num_26_full = 0 
                            den_26_full = 1
                    else: # either 14 or 52 week claim
                        num_26_full = 0
                        den_26_full = 0
                
        j += 1

    df_26['Num_26_full'] = [num_26_full]
    df_26['Den_26_full'] = den_26_full
    df_26['Num_26_path'] = [num_26_path]        
    df_26['Den_26_path'] = den_26_path
    df_26['Num_26_wrkast'] = [num_26_wrkast]
    df_26['Den_26_wrkast'] = den_26_wrkast
    return df_26

df_entry = g.get_group(451780)

f = get_26_outcome(df_entry, num_codes, den_codes, com_codes) # pass results to a new dataframe and append each time
f





                    


,Job_Seeker_ID,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,451780,1,1,0,0,0,0


# aggregation function for pre-quarter function:

In [52]:
# adds outcome to df_26
def agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes):
    df_26 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_26', 'Den_26']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome



    ids = df_pre["Job_Seeker_ID"].unique().astype("int64")
    for id in df_pre['Job_Seeker_ID'].unique():
        if math.isnan(id):
            pass
        else:
            df_entry = g.get_group(id)
            f = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
            df_26 = df_26.append(f)
            

    return df_26 # returns output from get_26_outcome 

In [81]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')
# job seekers
jobSeekers = jobSeekers["Job_Seeker_ID"].unique()


,Job_Seeker_ID,Num_26,Den_26,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,9386301003,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
0,8653691003,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
0,7870290004,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
0,6498977509,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
0,6638331004,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
0,5766520,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
0,2836891309,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
0,8771045004,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
0,615882709,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


### Pre-quarterisation 26 output file:

In [53]:
### Testing the aggregate function for 13 outcomes
df_26 = agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes)


warnings.filterwarnings('ignore')

print('Number of IDs for 26 week outcomes: ' + str(len(df_26)))

# output file: 

df_26.to_csv('C:/Users/AManalo/star_ratings_new/df_26_new.csv')


Number of IDs for 26 week outcomes: 4976


In [54]:
df_26['Den_26'].sum()

0

## Post-Quarterisation 26 week outcome
    - Not included if time to achieve an outcome is < 26 weeks. AKA commenced towards the end of the 2Y assessment period.
    - Using the quarter dates from the 13 week pre-quarterisation function 
    - maxNum = 1, maxDen = 8
    - Needs to commence before denominating.
    - If df_26 (pre-quarter), num=0 -> then only check the denominator
    - If df_26 (pre-quarter), num=1 -> then need to check when it numerates
    - If quarter falls outside of the 2Y period, then not included
Included: 
-	26 week Pathway Outcomes
-	26 week Full Outcome
o	With lodged, pending or approved status at the site 
-	26 week Bonus Outcomes/Work Assist 

Denominator: 
- In order to denominate: duration in assistance it at least 26 weeks - excluding periods of suspension.
- EXCEPTION: if ANCHOR_DATE = CLAIM_AVAILABLE_DATE (for 13 week outcome).
    Method: 
    1. If the claims in the current JobSeeker group are ONLY 26/52 week outcomes -> check for 13 week outcome in the df_supp. 
    2. Get 13 week claim (Full outcome) from df_supp. 
    3. Check if ANCHOR_DATE = CLAIM_AVAILABLE_DATE (commencement_date) -> then only add 13 weeks to the ANCHOR_DATE for 26 week time in duration. 
    4. If ANCHOR_DATE != CLAIM_AVAILABLE_DATE, then check duration in assistance is at least 26 weeks (excluding suspensions) - add 13 weeks to 13_week CLAIM_AVAILABLE_DATE. -> don't need to check suspensions for the 13 week claim, as the claim has already been lodged/pending/approved. 
    - Check suspensions accordingly. 

- doesn't denominate if exits before 26 week duration in assistance


In [55]:
targetStr = ['DES 26wk Full Bonus', 'DES 26wk Full Outcome', 'DES 26wk Pathway Outcome', 'DES 26wk Work Assist Outcome'] #valid outcomes
grp = g.get_group(168470)

grpList = set(grp.Claim_Rate_Type.values)

string_26 = next(i for i in targetStr if i in grpList)


Get the EXIT_DATE for the 26 wk claim -> to check whether job seeker exits before claim 

In [56]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]

startBounds = '2020-10-01'
endBounds = '2022-10-07'

def post_quarter_26(df_26, df_pre, g, g_supp, qtrs): 
    '''Takes the same arguments as post_quarter_13, however df_26 and df_supp are added. 
    g_supp is used to check the details of the 13 week claim outside of the 2Y assessment period.'''
    
    # max times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_26_full', 'Den_26_full','Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast']) # Dataframe template for result
    claimExists = False

    # looping through the pre-quarter 26 week outcome dataframe
    i = 0
    while i < len(df_26):
        js_id = df_26.Job_Seeker_ID.values[i]
        num_26_full = df_26['Num_26_full'].values[i]
        den_26_full = df_26['Den_26_full'].values[i]
        num_26_path = df_26['Num_26_path'].values[i]
        den_26_path = df_26['Den_26_path'].values[i]
        num_26_wrkast = df_26['Num_26_wrkast'].values[i]
        den_26_wrkast = df_26['Den_26_wrkast'].values[i]

        x = 0
        group = g.get_group(js_id)
        n = len(group)
        week_26 = pd.DateOffset(days=182)
        while x < n: # looping through entries of a group
            comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0])
            claim = group.Claim_Rate_Type.values[x]
            availDate = group.CLAIM_AVAILABLE_DATE.values[x]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[x])

            if (num_26_full == maxNum or den_26_full == maxDen) or (num_26_path == maxNum or den_26_path == maxDen) or (num_26_wrkast == maxNum or den_26_wrkast == maxDen):
                pass
            else:
                # check suspension
                period_26 = comDate + week_26
                if (group.SUSPENSION_DAYS.values[x] > 0) and (group.SUSPENSION_FROM_DATE.values[x] >= comDate) and (group.SUSPENSION_ACTUAL_END_DATE.values[x] <= period_26): 
                    sus_days = pd.DateOffset(days=int(group.SUSPENSION_DAYS.values[x]))
                    period_26 = comDate + week_26 + sus_days
                    # check exitDate
                if exitDate < period_26:
                    # don't include at all -> wipe all numerator/denominator:
                    num_26_full = 0
                    den_26_full = 0
                    num_26_path = 0
                    den_26_path = 0
                    num_26_wrkast = 0
                    den_26_wrkast = 0
                
                # loop through the quarters
                z = 0
                s = 0
                e = 1
                while z < len(qtrs):
                    start = pd.to_datetime(qtrs[z][s])
                    end = pd.to_datetime(qtrs[z][e])
                    if (den_26_full == maxDen) or (den_26_path == maxDen) or (den_26_wrkast == maxDen):
                        break
                    else:
                        # check every claim type and find which one to denominate on 
                        if num_26_full == 1 or num_26_path == 1 or num_26_wrkast == 1:
                            if num_26_full == 1:
                                # check whether duration is at least 26 weeks
                                if (comDate >= start) and (comDate <= end): # check if commencement date is in the current quarter ->
                                    # check if outcome is also in the current quarter
                                    if (availDate >= start) and (availDate <= end) and (availDate <= period_26): # available date is in the current commenced quarter -> then 
                                        den_26_full = 1
                                else: # commencement date has already passed
                                    if (availDate <= end) and (availDate >= start): # claim is available in the current quarter
                                        den_26_full += 1
                                    elif (availDate > end): #claim is in future quarter
                                        if (start >= period_26): # it has been at least 26 weeks then denominate
                                            den_26_full += 1
                                        else:
                                            den_26_full = 0
                                    else: # claim has already passed
                                        pass

                            elif num_26_path == 1:
                                # check whether duration is at least 26 weeks
                                if (comDate >= start) and (comDate <= end): # check if commencement date is in the current quarter ->
                                    # check if outcome is also in the current quarter
                                    if (availDate >= start) and (availDate <= end) and (availDate <= period_26): # available date is in the current commenced quarter -> then 
                                        den_26_path = 1
                                else: # commencement date has already passed
                                    if (availDate <= end) and (availDate >= start): # claim is available in the current quarter
                                        den_26_path += 1
                                    elif (availDate > end): #claim is in future quarter
                                        if (start >= period_26): # it has been at least 26 weeks then denominate
                                            den_26_path += 1
                                        else:
                                            den_26_path = 0
                                    else: # claim has already passed
                                        pass

                            elif num_26_wrkast == 1:
                                # check whether duration is at least 26 weeks
                                if (comDate >= start) and (comDate <= end): # check if commencement date is in the current quarter ->
                                    # check if outcome is also in the current quarter
                                    if (availDate >= start) and (availDate <= end) and (availDate <= period_26): # available date is in the current commenced quarter -> then 
                                        den_26_wrkast = 1
                                else: # commencement date has already passed
                                    if (availDate <= end) and (availDate >= start): # claim is available in the current quarter
                                        den_26_wrkast += 1
                                    elif (availDate > end): #claim is in future quarter
                                        if (start >= period_26): # it has been at least 26 weeks then denominate
                                            den_26_wrkast += 1
                                        else:
                                            den_26_wrkast = 0
                                    else: # claim has already passed
                                        pass

                        else: #none have numerators - check when these stop denominating 
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[0])
                            if den_26_full == 1:
                                if (comDate >= start) and (comDate <= end): # commenced in current quarter
                                    if (period_26 <= end): # if 26 weeks occurs in the same quarter then denominate:
                                        den_26_full = 1
                                    else:
                                        den_26_full = 0 # not eligible to start denominating
                                else: # already commenced
                                    if (period_26 <= start): # denominate 
                                        if (exitDate >= start) and (exitDate <= end):
                                            # exits in the current quarter 
                                            den_26_full += 1
                                            break
                                        else:
                                            den_26_full += 1


                            elif den_26_path == 1:
                                if (comDate >= start) and (comDate <= end): # commenced in current quarter
                                    if (period_26 <= end): # if 26 weeks occurs in the same quarter then denominate:
                                        den_26_path = 1
                                    else:
                                        den_26_path = 0 # not eligible to start denominating
                                else: # already commenced
                                    if (period_26 <= start): # denominate 
                                        if (exitDate >= start) and (exitDate <= end):
                                            # exits in the current quarter 
                                            den_26_path += 1
                                            break
                                        else:
                                            den_26_path += 1
                            
                            elif den_26_wrkast == 1:
                                if (comDate >= start) and (comDate <= end): # commenced in current quarter
                                    if (period_26 <= end): # if 26 weeks occurs in the same quarter then denominate:
                                        den_26_wrkast = 1
                                    else:
                                        den_26_wrkast = 0 # not eligible to start denominating
                                else: # already commenced
                                    if (period_26 <= start): # denominate 
                                        if (exitDate >= start) and (exitDate <= end):
                                            # exits in the current quarter 
                                            den_26_wrkast += 1
                                            break
                                        else:
                                            den_26_wrkast += 1
                            
                            else: # denominators are 0 
                                pass

                    z += 1    
            x += 1

        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_26_full' : num_26_full, 'Den_26_full' : den_26_full, 'Num_26_path' : num_26_path, 'Den_26_path' : den_26_path, 'Num_26_wrkast' : num_26_wrkast, 'Den_26_wrkast' : den_26_wrkast})
        df_post = df_post.append(data)
        i += 1
        
    return df_post

df_post_26 = post_quarter_26(df_26, df_pre, g, g_supp, qtrs)



### Running post_quarter_26 function


In [59]:

#df_post_26 = df_post_26.drop(columns=['Num_13', 'Den_13'])


2496.0

In [60]:
qtrs

[['2020-10-01', '2020-12-31'],
 ['2021-01-01', '2021-03-31'],
 ['2021-04-01', '2021-06-30'],
 ['2021-07-01', '2021-09-30'],
 ['2021-10-01', '2021-12-31'],
 ['2022-01-03', '2022-03-31'],
 ['2022-04-01', '2022-06-30'],
 ['2022-07-01', '2022-09-30']]

In [61]:
df_post_26.to_csv('C:/Users/AManalo/star_ratings_new/df_post_26.csv')

### Separating the performance measures 26 outcomes 

## 52 week outcomes

### Pre-quarterisation function (52 week outcome)

In [62]:
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')



startBounds = '2020-10-01'
endBounds = '2022-10-07'

In [63]:

def get_52_outcome(jobSeekerEntry, num_codes, den_codes, com_codes): # takes a row (groupBy on Job_Seeker_ID, list of eligible numerator phase codes + denominator phase codes
    # sort jobSeekerEntry by COMMENCEMENT_DATE
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry # group containing the attributes for one job_seeker_ID
    id = group.index.values.astype(int)[0]
    # initialise numerator and denominator 
    num_52 = 0
    den_52 = 0
    df_52 = pd.DataFrame()
    # place id into dataframe
    df_52['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    week_52 = pd.DateOffset(days = 182) # 52 weeks -> 26 + 26 = 52 weeks -> days
    eduStr = 'DES 52wk Full Outcome Education'
    # Check number of entries in group
    targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
    num_entries = int(len(df_entry))

    valid_claims = ['Approved', 'Pending', 'Lodged'] 
    exitDate = group.EXIT_DATE.values[0] # if exit date exists ->  exit date is the same for all entries

    if num_entries == 1: # if one -> check for 52 week full outcome 
        code_exists = group.PLACE_REF.values[0] in num_codes

        # need to check if participant is in the EA or PPS phase at the site (at least 52 weeks (excluding periods of suspension))
        # if participant is NOT in assistance for at least 52 weeks - excluding suspension periods

        if (group.Claim_Rate_Type.values[0] in targetStr) and (group.Claim_Rate_Type.values[0].isin(eduStr) == False) and (group.Claim_Status.values[0] in valid_claims) and (code_exists == True): #NOT EDUCATION
            comDate = group.COMMENCEMENT_DATE.values[0] # get the first commencement date
            if group.SUSPENSION_DAYS.values[0] > 0: #excludes periods of suspension 
                sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[0]))
                period_52 = comDate + week_52 + sus_days
                if (period_52 <= '2020-10-01') or (period_52 >= '2022-10-07'):
                    num_52 = 0 
                    den_52 = 0 # out of bounds
                else:        
                    # check if exit date is less than period_52 (with suspension period)
                    if exitDate < period_52:
                        num_52 = 0
                        den_52 = 0
                
            if (group.CLAIM_AVAILABLE_DATE.values[1] >= period_52): # been AT LEAST 52 weeks 
                if (period_52 <= '2020-10-01') or (period_52 >= '2022-10-07'):
                    num_52 = 0 
                    den_52 = 0 # out of bounds
                else:
                    num_52 += 1
                    den_52 += 1

            df_52['Num_52'] = num_52
            df_52['Den_52'] = den_52
            return df_52              

        
        if (group.OUTCOME_TYPE.values[0] == 'Education'): # we don't include the Education outcomes     
            num_52 = 0 
            den_52 = 0 
           
            
    else:
        # multiple entries in group. Loop through entries:
        
        j = 0
        # not comparing the numbers of the placement_seq_num as these have already been sorted, so have the dates
      
        comDate = group.COMMENCEMENT_DATE.values[0]
        exitDate = group.EXIT_DATE.values[0] 
        while j < len(group.COMMENCEMENT_DATE.values):
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            # continue:
            code_exists = group.PLACE_REF.values[j] in num_codes
            if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Rate_Type.values[j] != eduStr) and (group.Claim_Status.values[j] in valid_claims) and (code_exists == True):
                num_52 += 1 # numerates
                den_52 = 1
                
                df_52['Num_52'] = [num_52]
                df_52['Den_52'] = den_52
                return df_52
                
            else: 
                commenced = group.PLACE_REF.values[j] in com_codes
                if commenced:
                    # get commencement date -> 
                    comDate = group.COMMENCEMENT_DATE.values[j] # first commencement date - of a participant commencing at a site
                    # add 52 weeks to the commencement date -> then check how long it takes to TRN or EXT (excluding suspensions) 
                        # if no outcome is recorded -> then participant denominates
                    if group.SUSPENSION_DAYS.values[j] == 0: # so no suspension ->
                        period_52 = week_52 + comDate
                        period_52 = pd.to_datetime(period_52)
                   
                    else: #has a suspension
                        period_52 = comDate + week_52 + timedelta(days=int(group.SUSPENSION_DAYS.values[j]))
                        period_52 = pd.to_datetime(period_52)
                    
                    if group.CLAIM_AVAILABLE_DATE.values[j] >= period_52:
                        # check if outcome is recorded using claim_available_date:
                        if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Rate_Type.values[j] != eduStr) and (group.Claim_Status.values[j] in valid_claims) and (code_exists == True):
                            startBounds = pd.to_datetime('2020-10-01')
                            endBounds = pd.to_datetime('2022-10-07')
                            if (period_52 <= startBounds) or (period_52 >= endBounds):
                                num_52 = 0 
                                den_52 = 0 # out of bounds
                            else:
                                num_52 = 1
                                den_52 = 1
                                
                            df_52['Num_52'] = num_52
                            df_52['Den_52'] = den_52
                            
                            return df_52
                            
                        else:
                            startBounds = pd.to_datetime('2020-10-01')
                            endBounds = pd.to_datetime('2022-10-07')
                            if (period_52 <= startBounds) or (period_52 >= endBounds): # period_52 is not within the 2 year assessment period 
                                num_52 = 0 
                                den_52 = 0 # out of bounds
                            else: 
                                # no claim exists -
                                num_52 = 0
                                den_52 = 1
                                
                            df_52['Num_52'] = num_52
                            df_52['Den_52'] = den_52


                    else:
                        # Check if there is NO 52 claims anywhere:
                        try: 
                            targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
                            grpList = set(group.Claim_Rate_Type.values)
                            string_52 = next(i for i in targetStr if i in grpList) # the 52 week claim type that exists in the current group
                            if string_52 is None:
                                num_52 = 0
                                # check whether 52 weeks has passed comDate and is in the 2Y period 
                                if (period_52 >= startBounds) and (period_52 <= endBounds): # valid date - but no 52 outcome recorded
                                    den_52 = 1 
                                
                                else:
                                    den_52 = 0
                                
                                df_52['Num_52'] = num_52
                                df_52['Den_52'] = den_52
                                break
                            else: 
                                pass
                        except StopIteration as e:
                            break
                        
                            

                            
                    
                else:
                    # code is not COM -> check if commencement_date of this code is <= 52 week period

                    # check if code is SUS_EA: (Counted in denominator if completed more than 52 weeks with us. Includes suspension period)
                    if group.PLACE_REF.values[j] == 'SUS_EA':
                        period_52 = group.COMMENCEMENT_DATE.values[0] + week_52 # get commencement date of first case
                        period_52 = pd.to_datetime(period_52)
                      
                        if group.CLAIM_AVAILABLE_DATE.values[j] >= period_52:
                            if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Status.values[j] in valid_claims):
                                startBounds = pd.to_datetime('2020-10-01')
                                endBounds = pd.to_datetime('2022-10-07')
                                if (period_52 <= startBounds) or (period_52 >= endBounds):
                                    num_52 = 0 
                                    den_52 = 0 # out of bounds
                                else:
                                    # then doesn't denominate: 
                                    num_52 = 1
                                    den_52 = 1

                                df_52['Num_52'] = num_52
                                df_52['Den_52'] = den_52
                                return df_52
                                
                            else:
                                startBounds = pd.to_datetime('2020-10-01')
                                endBounds = pd.to_datetime('2022-10-07')
                                if (period_52 <= startBounds) or (period_52 >= endBounds):
                                    num_52 = 0 
                                    den_52 = 0 # out of bounds
                                
                                   
                                df_52['Num_52'] = num_52
                                df_52['Den_52'] = den_52
                                
                                    

                    else:
                        # TRN or EXT
                        # check if TRN or EXT before 52 week period (excluding suspension)
                        period_52 = group.COMMENCEMENT_DATE.values[0] + week_52
                        period_52 = pd.to_datetime(period_52)
                        if group.SUSPENSION_DAYS.values[j] is pd.NaT: # so no suspension ->
                            period_52 = group.COMMENCEMENT_DATE.values[0] + week_52
                            period_52 = pd.to_datetime(period_52)
                        else: 
                            period_52 = group.COMMENCEMENT_DATE.values[0] + week_52 + timedelta(days=int(group.SUSPENSION_DAYS.values[j])) # current suspension_DAYS value
                            period_52 = pd.to_datetime(period_52)
                        # check if numerate:
                        if group.CLAIM_AVAILABLE_DATE.values[j] >= period_52:
                            if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Rate_Type.values[j] != eduStr) and (group.Claim_Status.values[j] in valid_claims):
                                startBounds = pd.to_datetime('2020-10-01')
                                endBounds = pd.to_datetime('2022-10-07')
                                if (period_52 <= startBounds) or (period_52 >= endBounds):
                                    num_52 = 0 
                                    den_52 = 0 # out of bounds
                                else:
                                    num_52 = 1
                                    den_52 = 1

                                df_52['Num_52'] = num_52
                                df_52['Den_52'] = den_52
                                return df_52
                            
                            else:
                                startBounds = pd.to_datetime('2020-10-01')
                                endBounds = pd.to_datetime('2022-10-07')
                                if (period_52 <= startBounds) or (period_52 >= endBounds):
                                    num_52 = 0 
                                    den_52 = 0 # out of bounds
                        

                                df_52['Num_52'] = num_52
                                df_52['Den_52'] = den_52
                                

                                
                        else: # check if participant ext or tfn before 52 week
                            if pd.to_datetime(group.EXIT_DATE.values[j]) <= period_52: # exited before 52 weeks                                
                                den_52 = 0
                                num_52 = 0
                                df_52['Num_52'] = [num_52]
                                df_52['Den_52'] = [den_52]
                            else:  # exited after 52 weeks
                                num_52 = 0
                                den_52 = 1
                                df_52['Num_52'] = [num_52]
                                df_52['Den_52'] = [den_52]
                                
                            
                

                
            j += 1
        
    df_52['Num_52'] = num_52
    df_52['Den_52'] = den_52    
    
    return df_52 # returns dataframe with 52 week numerator and denominator


f = get_52_outcome(df_entry, num_codes, den_codes, com_codes) # pass results to a new dataframe and append each time
f

,Job_Seeker_ID,Num_52,Den_52
0,451780,1,1


# aggregation function for pre-quarter function:

In [64]:
# adds outcome to df_52
def agg_outcomes_52(df_pre, g, num_codes, den_codes, com_codes):
    df_52 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_52', 'Den_52']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome



    ids = df_pre["Job_Seeker_ID"].unique().astype("int64")
    for id in df_pre['Job_Seeker_ID'].unique():
        if math.isnan(id):
            pass
        else:
            df_entry = g.get_group(id)
            f = get_52_outcome(df_entry, num_codes, den_codes, com_codes)
            df_52 = df_52.append(f)
            

    return df_52 # returns output from get_52_outcome 

### Pre-quarterisation 52 output file:

In [65]:
### Testing the aggregate function for 13 outcomes
df_52 = agg_outcomes_52(df_pre, g, num_codes, den_codes, com_codes)


warnings.filterwarnings('ignore')

print('Number of IDs for 52 week outcomes: ' + str(len(df_52)))


df_52.to_csv('C:/Users/AManalo/star_ratings_new/df_52_new.csv')

Number of IDs for 52 week outcomes: 4976


### Separating the performance measures 52 outcomes 

In [66]:
df_52_new = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path', 'Num_52_wrkast', 'Den_52_wrkast'])

for index, row in df_52.iterrows():
    
    g_key = row['Job_Seeker_ID']
    try:
        g_group = g_supp.get_group(g_key)
    except KeyError as e:
        print(str(e))
        pass
    g_claims = g_group.CLAIM_RATE_TYPE.values
    if 'DES 52wk Full Outcome' in g_claims:
        #   get the relevant dataframe matches
            row_match = df_52[df_52['Job_Seeker_ID'] == g_key]
            Num_52_old = row_match['Num_52']
            Den_52_old = row_match['Den_52']
            data = pd.DataFrame({'Job_Seeker_ID' : [g_key], 'Num_52_full': Num_52_old, 'Den_52_full' : Den_52_old})
            df_52_new = df_52_new.append(data)
             
    elif 'DES 52wk Pathway Outcome' in g_claims:
            #   get the relevant dataframe matches
            row_match = df_52[df_52['Job_Seeker_ID'] == g_key]
            Num_52_old = row_match['Num_52']
            Den_52_old = row_match['Den_52']
            data = pd.DataFrame({'Job_Seeker_ID' : [g_key], 'Num_52_path': Num_52_old, 'Den_52_path' : Den_52_old})
            df_52_new = df_52_new.append(data) 

df_52_new.to_csv('df_52_new.csv') 

## Post-Quarterisation 52 week outcome

In [67]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]

startBounds = '2020-10-01'
endBounds = '2022-10-07'

def post_quarter_52(df_52, df_pre, g, g_supp, qtrs): 
    '''Takes the same arguments as post_quarter_26, however df_52 and df_supp are added. 
    g_supp is used to check the details of the 26 week claim outside of the 2Y assessment period.'''
    
    # max times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_26', 'Den_26']) # Dataframe template for result
    claimExists = False

    # looping through the pre-quarter 52 week outcome dataframe
    i = 0
    while i < len(df_52):
        js_id = df_52.Job_Seeker_ID.values[i]
        num_52 = df_52.Num_52.values[i]
        den_52 = df_52.Den_52.values[i]

        # if no 52 week outcome:
        if (num_52 == 0) and (num_52 == den_52): # if 0/0 -> 52 outcome is not found in 2Y period
            Num_52 = 0
            Den_52 = 0
            data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_52' : Num_52, 'Den_52' : Den_52})
            df_post.append(data)
            # find number of times that participant denominates 
            pass #already added data into dataframe for this case
        else: 
            # Num_52 = 0 / 1 or Num_52 = 1 / 1 
            x = 0
            Num_52 = 0 
            Den_52 = 0 
            group = g.get_group(js_id)
            g_supp1 = g_supp.get_group(js_id)
            n = len(group)
            
            week_26 = pd.DateOffset(days = 182) # add 26 weeks to 26-wk-outcome claim_available_date
        
        # Get the appropriate details from the current job seeker group ->
    
            comDate = group.COMMENCEMENT_DATE.values[0] # from this, depending on each case -> the time in duration could be 52/26 weeks
            claimList = group.Claim_Rate_Type.values #get list of all claims -> to check if 26 week is here
           
           # update availDate and exitDate -> once the 52 week outcome is found
            availDate = group.CLAIM_AVAILABLE_DATE.values[x] # get avail date from 52 outcome (if exists), otherwise use period_52 -> calculate this
            exitDate = pd.to_datetime(group.EXIT_DATE.values[x])


            target_52 = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
            valid_claims = ['Approved', 'Pending', 'Lodged'] 
            claim_26 = 'DES 26wk Full Outcome'
            
            # finding the 26 wk claim to determine period_52 -> date at which the 52 week outcome is due 
            # 26 week claim is IN the CURRENT 2Y period
            if claim_26 in group.Claim_Rate_Type.values: # 26 week claim is in 2Y period -> just add 26 weeks to claim_available_date
                ind_26 = (group.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax() 
                # then get the claim_avaialble_date for 26 week outcome -> use as comDate
                comDate = group.CLAIM_AVAILABLE_DATE.values[ind_26]
                # check if 52 week outcome exists 
                try:
                    targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
                    grp = group # reassinging the group variable 
                    grpList = set(grp.Claim_Rate_Type.values)
                    string_52 = next(i for i in targetStr if i in grpList) # returns outcome if exists
                    if (string_52 == "") or (string_52 == None): # no 52 week outcome exists
                        # get the suspension dates (get from last argument of the group) 
                        gLen = len(grp.values) #length of the group
                        if (grp.SUSPENSION_FROM_DATE.values[gLen - 1].isna() == False) and (grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1].isna() == False): #so suspension exists
                            start_sus = grp.SUSPENSION_FROM_DATE.values[gLen - 1]
                            end_sus = grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1]
                            # check if suspension dates occur AFTER the 26 week claim, therefore affecting the 52 week claim comDate, AKA check that suspension occurs between 26 week claim avail date + 26 weeks period
                            due_52 = comDate + pd.DateOffset(days = 91) # 52 claim eligible/due date
                            if (start_sus >= comDate) and (end_sus <= due_52):
                                # get suspension days
                                sus_days = grp.SUSPENSION_DAYS.values[gLen - 1]
                                # calculate period_52 ->
                                period_52 = comDate + sus_days
                                period_52 = pd.to_datetime(period_52)  
                            else:
                                period_52 = due_52 # no suspension exists
                                period_52 = pd.to_datetime(period_52) 
                        else:
                            # suspension dates are missing
                            period_52 = comDate + pd.DateOffset(days = 91)
                            period_52 = pd.to_datetime(period_52) 
                    else: # 52 week outcome exists -> suspension exists -> take period_52 as due date of the 52 week outcome , still need to calculate the suspension period 
                        ind_52 = (grp.Claim_Rate_Type.values == string_52).argmax() # index of the 52 week outcome
                        due_52 = (grp.CLAIM_AVAILABLE_DATE.values[ind_52]) # even if there are suspensions -> claim_available_date already accounts for this.
                        period_52 = due_52 
                        period_52 = pd.to_datetime(period_52) 

                except StopIteration as e:
                    break 


                    
                
            else: # claim_26 is outside of the 2Y period OR claim_26 doesn't exist
                # look at df_supp to get the 26 week claim
                if claim_26 in g_supp1.CLAIM_RATE_TYPE.values: #26 week claim exists in the 6 months prior to 2Y period
                    ind_26 = (g_supp1.CLAIM_RATE_TYPE.values == 'DES 26wk Full Outcome').argmax()
                    # then get the claim_available_date for the 26w week outcome -> use as comDate
                    comDate = g_supp1.CLAIM_AVAILABLE_DATE.values[ind_26]
                    # find period_52 from the claim_26 prior to the 2Y period -> see if period_52 is in the 2Y period
                    # find suspensions - suspension for 52 week 
                    # check if 52 week outcome exists
                 
                
                    targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
                    grp = g_supp1 # reassinging the group variable 
                    grpList = set(grp.CLAIM_RATE_TYPE.values)
                    string_52 = next((i for i in targetStr if i in grpList), "") # returns outcome if exists
               
                    if (string_52 == "") or (string_52 == None): # no 52 week outcome exists
                        # get the suspension dates (get from last argument of the group) 
                        gLen = len(grp.values) #length of the group
                        if (grp.SUSPENSION_FROM_DATE.values[gLen - 1].isna() == False) and (grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1].isna() == False): #so suspension exists
                            start_sus = grp.SUSPENSION_FROM_DATE.values[gLen - 1]
                            end_sus = grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1]
                            # check if suspension dates occur AFTER the 26 week claim, therefore affecting the 52 week claim comDate, AKA check that suspension occurs between 26 week claim avail date + 26 weeks period
                            due_52 = comDate + pd.DateOffset(days = 91) # 52 claim eligible/due date
                            if (start_sus >= comDate) and (end_sus <= due_52):
                                # get suspension days
                                sus_days = grp.SUSPENSION_DAYS.values[gLen - 1]
                                # calculate period_52 ->
                                period_52 = due_52 + sus_days
                                period_52 = pd.to_datetime(period_52)  
                            else:
                                period_52 = due_52 # no suspension exists
                                period_52 = pd.to_datetime(period_52) 
                        else:
                            # suspension dates are missing
                            period_52 = comDate + pd.DateOffset(days = 91)
                    else: # 52 week outcome exists -> suspension exists -> take period_52 as due date of the 52 week outcome , still need to calculate the suspension period 
                        ind_52 = (grp.CLAIM_RATE_TYPE.values == string_52).argmax() # index of the 52 week outcome
                        due_52 = (grp.CLAIM_AVAILABLE_DATE.values[ind_52]) # even if there are suspensions -> claim_available_date already accounts for this.
                        period_52 = due_52
                        period_52 = pd.to_datetime(period_52) 
                else: # no 26 week claim ->
                    # need to calculate period_26 from COMMENCEMENT_DATE if the participant commences
                    targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
                    grp = g_supp1 # reassinging the group variable 
                    grpList = set(grp.CLAIM_RATE_TYPE.values)
                    string_52 = next((i for i in targetStr if i in grpList), "") # returns outcome if exists
                    # get commencement date of participant
                    comDate = grp.COMMENCEMENT_DATE.values[0]
                    gLen = len(grp.values) #length of the group
                    # check if exit date exists
                    if grp.EXIT_DATE.values[0] == "":
                        # check if suspension exists
                        if (grp.SUSPENSION_FROM_DATE.values[gLen - 1].isna() == False) and (grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1].isna() == False): #so suspension exists
                            start_sus = grp.SUSPENSION_FROM_DATE.values[gLen - 1]
                            end_sus = grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1]
                            due_52 = comDate + pd.DateOffset(days = 91) # 52 claim eligible/due date
                            # check if suspension dates occur AFTER the 26 week claim, therefore affecting the 52 week claim comDate, AKA check that suspension occurs between 26 week claim avail date + 26 weeks period
                            if (start_sus >= comDate) and (end_sus <= due_52):
                                # get suspension days
                                sus_days = grp.SUSPENSION_DAYS.values[gLen - 1]
                                # calculate period_52 ->
                                period_52 = due_52 + sus_days
                                period_52 = pd.to_datetime(period_52)  
                            else: # suspension out of relevant 52 week bounds
                                period_52 = due_52 # no suspension exists
                                period_52 = pd.to_datetime(period_52) 
                            
                            
                        else: # no suspensions 
                            due_52 = comDate + pd.DateOffset(days=91)
                    else:
                        # Check suspensions 
                        if (pd.isna(grp.SUSPENSION_FROM_DATE.values[gLen - 1]) == False) and (pd.isna(grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1]) == False): #so suspension exists
                            start_sus = grp.SUSPENSION_FROM_DATE.values[gLen - 1]
                            end_sus = grp.SUSPENSION_ACTUAL_END_DATE.values[gLen - 1]
                            due_52 = comDate + pd.DateOffset(days = 91) # 52 claim eligible/due date
                            # check if suspension dates occur AFTER the 26 week claim, therefore affecting the 52 week claim comDate, AKA check that suspension occurs between 26 week claim avail date + 26 weeks period
                            if (start_sus >= comDate) and (end_sus <= due_52):
                                # get suspension days
                                sus_days = grp.SUSPENSION_DAYS.values[gLen - 1]
                                # calculate period_52 ->
                                period_52 = due_52 + sus_days
                                period_52 = pd.to_datetime(period_52)  
                            else: # suspension out of relevant 52 week bounds
                                period_52 = due_52 # no suspension exists
                                period_52 = pd.to_datetime(period_52) 
                            
                            
                        else: # no suspensions 
                            due_52 = comDate + pd.DateOffset(days=91)

                       
                       


            targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
            grp = g_supp1 # reassinging the group variable 
            grpList = set(grp.CLAIM_RATE_TYPE.values)
            string_52 = next((i for i in targetStr if i in grpList), "") # returns outcome if exists
            # loop through the quarters ->
            z = 0
            s = 0
            e = 1
            while z < len(qtrs):
                start = pd.to_datetime(qtrs[z][s])
                end = pd.to_datetime(qtrs[z][e])
                if Num_52 == maxNum or Den_52 == maxDen: #num = 1, den = 8 (break)
                    break
                else:
                    
                    # using string_52 (to see if 52 week claim is achieved) and period_52
                    if (string_52 == "") or (string_52 == None): #claim doesn't exist 
                        # check whether the job seeker commences in the current quarter or previously 
                        # IN ORDER TO DEN -> (1) comDate has passed, AND (2) period_52 has passed
                        # check -> WHETHER PARTICIPANT EXITS 
                        if (start<=comDate<=end) or (comDate <= start):
                            # check if participant has EXITED before period_52: ASSUMPTION -> EXIT_DATE is the same across all entries of the group
                            exit_date = pd.to_datetime(group.EXIT_DATE.values[0])
                            if (exit_date <= period_52): #EXITED before period_52
                                Num_52 = 0
                                Den_52 = 0
                                break # return the result
                            else:
                                # check if period_52 is achieved
                                if (start<=period_52<=end): #period_52 in the current quarter
                                    # since no 52 outcome recorded -> denominates
                                    Num_52 = 0
                                    Den_52 += 1
                                elif (period_52 <= start): # in a previous quarter -> then denominate
                                    Den_52 += 1
                        else:
                            Den_52 = 0 #participant has not commenced
                    else: #52 week outcome exists
                        # check whether the job seeker commences in the current quarter or previously 
                        # IN ORDER TO DEN -> (1) comDate has passed, AND (2) period_52 has passed
                        # check -> WHETHER PARTICIPANT EXITS 
                        if (start<=comDate<=end) or (comDate <= start):
                            # check if participant has EXITED before period_52: ASSUMPTION -> EXIT_DATE is the same across all entries of the group
                            if (pd.to_datetime(group.EXIT_DATE.values[0]) <= period_52): #EXITED before period_52
                                Num_52 = 0
                                Den_52 = 0
                                break # return the result
                            else:
                                # check if period_52 is achieved
                                if (start<=period_52<=end): #period_52 in the current quarter
                                    # 52 outcome recorded -> finish denominating and record a numerator -> break loop
                                    Num_52 = 1
                                    Den_52 += 1
                                    break # 52 outcome already found
                        else:
                            Den_52 = 0 #participant has not commenced
                z += 1

        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_52' : Num_52, 'Den_52' : Den_52})
        df_post = df_post.append(data)

        i += 1
    
    return df_post


### Running the post_quarter_52 week function

In [68]:
df_post_52 = post_quarter_52(df_52, df_pre, g, g_supp, qtrs)

df_post_52 = df_post_52.drop(columns=['Num_26', 'Den_26'])
# output file
df_post_52.to_csv('C:/Users/AManalo/star_ratings_new/df_post_52.csv')


In [69]:
len(df_pre.Claim_Rate_Type.values == 'DES 52wk Full Outcome')
df_post_52

,Job_Seeker_ID,Num_52,Den_52
0,9386301003,0.0,8.0
0,8653691003,0.0,8.0
0,7870290004,0.0,0.0
0,6498977509,0.0,0.0


### Separating the post-quarter 52 week outcomes - subtypes

In [70]:
df_52_newPost = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'])

for index, row in df_post_52.iterrows():
    
    g_key = row['Job_Seeker_ID']
    g_group = g_supp.get_group(g_key)
    g_claims = g_group.CLAIM_RATE_TYPE.values
    if 'DES 52wk Full Outcome' in g_claims:
        #   get the relevant dataframe matches
            row_match = df_post_52[df_post_52['Job_Seeker_ID'] == g_key]
            Num_52_old = row_match['Num_52']
            Den_52_old = row_match['Den_52']
            data = pd.DataFrame({'Job_Seeker_ID' : [g_key], 'Num_52_full': Num_52_old, 'Den_52_full' : Den_52_old})
            df_52_newPost = df_52_newPost.append(data)
             
    elif 'DES 52wk Pathway Outcome' in g_claims:
            #   get the relevant dataframe matches
            row_match = df_post_52[df_post_52['Job_Seeker_ID'] == g_key]
            Num_52_old = row_match['Num_52']
            Den_52_old = row_match['Den_52']
            data = pd.DataFrame({'Job_Seeker_ID' : [g_key], 'Num_52_path': Num_52_old, 'Den_52_path' : Den_52_old})
            df_52_newPost = df_52_newPost.append(data) 

df_52_newPost.to_csv('C:/Users/AManalo/star_ratings_new/df_52_post.csv') # replace the old df_52_post

In [71]:
df_52_newPost

,Job_Seeker_ID,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,6498977509,0.0,0.0,NaN,NaN



#### Caseload/Missing
- Caseload - identify low job seekers (<20 caseload) -> insufficient load -> identify contracts
- Count case load -> overall and let depar know - done
- missing data -> gaps we need to fill - done
- Identify people that are not getting an outcome at all - done
- Find proportion of data that needs to be cleaned (%)  - done



1) Caseload File


In [ ]:
# total caseload:

#naVal = df_cases.isna().sum() * 100 / len(df_cases)
#missValDf = pd.DataFrame({'column_name': df_cases.columns, 'percent_missing': naVal})

#print('total cases: ' + str(len(df_cases)))
#missValDf.sort_values(by=['percent_missing'], ascending=False).head(17)



2) Claims File

In [ ]:

# naVal = df_claims.isna().sum() * 100 / len(df_claims)
# missValDf = pd.DataFrame({'column_name': df_claims.columns, 'percent_missing': naVal})

# print('total claims: ' + str(len(df_claims)))
# missValDf.sort_values(by=['percent_missing'], ascending=False).head(3)



### Participants with no outcome recorded for Dec Star Rating 2021 - Job_Seeker_ID

In [ ]:
# # participants without an outcome - get the ones with an outcome from the 'find' variable
# outcome = df_pre["Job_Seeker_ID"].drop_duplicates()
# df_pre_temp = df_pre_temp["Job_Seeker_ID"].drop_duplicates()
# vals = df_pre_temp.isin(outcome)
# # list of participants (Job_Seeker_ID) without an outcome

# # getting the false values:
# no_outcome = (vals != True).index.values



### Proportion of Data that needs Cleaning
1. Listing distinct values for each attribute and inspecting manually

In [ ]:
df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Code', 'Site_Name',
       'ESA_Code', 'ESA Name', 'State', 'specialist_site_type_code',
       'COMMENCEMENT_DATE', 'EXIT_DATE', 'Claim_ID', 'Claim_Rate_Type',
       'Claim_Status', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE',
       'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE',
       'OUTCOME_TYPE', 'SUSPENSION_DAYS', 'PLACE_REF'],
      dtype='object')

In [ ]:
# place_seq_num = df_pre['PLACEMENT_SEQUENCE_NUMBER'].drop_duplicates().values
# print('place_seq_num: ', np.sort(place_seq_num))

# job_seeker_id = df_pre['Job_Seeker_ID'].drop_duplicates().values
# #print('job_seekers: ', job_seeker_id)

# place_stat_cd = df_pre['PLACEMENT_STATUS_CODE'].drop_duplicates().values
# print('place_stat_cd: ', place_stat_cd)

# ref_phase_cd = df_pre['REFERRAL_PHASE_CODE'].drop_duplicates().values
# print('ref phase code: ', ref_phase_cd)

# commence_dt = df_pre['COMMENCEMENT_DATE'].drop_duplicates().values
# #print('commence_date: ', commence_dt)

# claim_id = df_pre['Claim_ID'].drop_duplicates().values
# print('claim_id: ', claim_id)

# claim_rate = df_pre['Claim_Rate_Type'].drop_duplicates().values
# print('claim_rate: ', claim_rate)

# claim_stat = df_pre['Claim_Status'].drop_duplicates().values
# print('claim stat: ', claim_stat)

# claim_start = df_pre['CLAIM_START_DATE'].drop_duplicates().values
# #print('claim_start: ', claim_start)

# claim_avail = df_pre['CLAIM_AVAILABLE_DATE'].drop_duplicates().values
# #print('claim_avail: ', claim_avail)

# claim_end = df_pre['CLAIM_END_DATE'].drop_duplicates().values
# #print('claim_end: ', claim_end)

# suspen_start = df_pre['SUSPENSION_FROM_DATE'].drop_duplicates().values
# #print('suspen_start_date: ', suspen_start)

# suspen_end = df_pre['SUSPENSION_ACTUAL_END_DATE'].drop_duplicates().values
# #print('suspen end: ', suspen_end)

# outcome_type = df_pre['OUTCOME_TYPE'].drop_duplicates().values
# print('outcome_type: ', outcome_type)



- Missing values for placement_sequence_number: 13, 15, 16, 17, 18, 19
- Min: 1, Max: 20

- Incorrect spelling for claim_rate: 'Post Placement Special Payment - DES DMS Job Seeke' - 'Job Seeker?'

# Testing

In [74]:
import io, os, sys, types
import unittest
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

### Coffs Harbour Test for 26 week outcomes
Contract: 0212826F

In [75]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')
# job seekers
jobSeekers = jobSeekers["Job_Seeker_ID"].unique()

len(jobSeekers)

13

In [80]:
class TestNotebook(unittest.TestCase):
    # 13 week test cases:                                                                      
    def test_edu_13(self):
        '''If DES 13wk Full Outcome Education = Claim_Rate_Type, then den=1, num=0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome Education' in df_entry.Claim_Rate_Type.values:
                out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                num = out['Num_13'][0]
                den = out['Den_13'][0]

                assert (num == 0), "Education 13 outcome should NOT numerate."
                assert (den == 1), "Education 13 outcome MUST denominate."

    def test_valid_13(self):
        '''Claim status should be Approved, Lodged, Pending for it to numerate and claim_available_date AFTER or EQUAL TO commencement_date'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                if df_entry.Claim_Status.values[ind_13] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_13] >= df_entry.COMMENCEMENT_DATE.values[ind_13]):
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 1), "Valid 13 full outcome, must numerate."
                    assert (den == 1), "Valid 13 full outcome, must denominate."

                else: # Claim Status is not valid
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "Invalid 13 full outcome, doesn't numerate."
                    assert (den == 1), "Invalid 13 full outcome, must denominate."



    def test_13_date(self):
        '''If 13 week claim available date is BEFORE commencement date -> then 0/0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                if df_entry.CLAIM_AVAILABLE_DATE.values[ind_13] < df_entry.COMMENCEMENT_DATE.values[ind_13]:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "Invalid Claim_Available_Date, doesn't numerate."
                    assert (den == 0), "Invalid Claim_Available_Date, doesn't denominate."

    def test_13_exit(self):
        '''If a participant exits before 13 week claim is due, then Num=0,Den=0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                claim_avail_date = pd.to_datetime(df_entry.CLAIM_AVAILABLE_DATE.values[ind_13])
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_13])
                if exit_date < claim_avail_date:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]
                    try: 
                        assert (num == 0), "Exits before claim, doesn't numerate."
                        assert (den == 0), "Exits before claim, doesn't denominate."
                    except AssertionError as e:
                        print(e)
                        print(df_entry.Job_Seeker_ID)
                    


    def test_13_path(self):
        '''If a 13 week pathway outcome is recorded, then Num=0, Den=1. Participant denominates.'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                pass
            else:
                if 'DES 13wk Pathway Outcome' in df_entry.Claim_Rate_Type.values:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "13 wk pathway doesn't numerate."
                    assert (den == 1), "13 wk pathway must denominate."
    
    ###### 26 week outcome tests
    def test_26_edu(self):
        '''26 week Education outcomes are not included. Num=0, Den=1'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Full Outcome Education' in df_entry.Claim_Rate_Type.values:
                out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                num = out['Num_26_full'][0]
                den = out['Den_26_full'][0]
                assert (num == 0), "Education 26 outcome not valid. Doesn't numerate."
                assert (den == 1), "Education 26 outcome included in the denominator."

    def test_26_valid_path(self):
        '''26 week Pathway outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses.'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Pathway Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Pathway Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 1), "Valid 26 pathway, must numerate."
                    assert (den == 1), "Valid 26 pathway, must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid 26 pathway, doesn't numerate."
                    assert (den == 1), "Invalid 26 pathway, must denominate."

    def test_26_valid_full(self):
        '''26 week Full outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses.'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 1), "Valid 26 outcome, must numerate." 
                    assert (den == 1), "Valid 26 outcome, must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid claim status, doesn't numerate."
                    assert (den == 1), "Invalid claim status, must denominate."
            

    def test_26_valid_bonus(self):
        '''26 week Bonus or 26 week work assist outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Bonus' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Bonus').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26_wrkast'][0]
                    assert (num == 1), "Valid bonus claim, must numerate." 
                    assert (den == 1), "Valid bonus claim, must denominate."

                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid claim status, should not numerate."
                    assert (den == 1), "Invalid claim status, must denominate."
                
            elif 'DES 26wk Work Assist Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Work Assist Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26)_wrkast'][0]

                    assert (num == 1), "Valid 26wk Work Assist must numerate."
                    assert (den == 1), "Valid 26wk Work Assist must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26_wrkast'][0]

                    assert (num == 0), "Invalid 26 work assist claim status, no outcome."
                    assert (den == 1), "Invalid 26 work assist claim status, must denominate."

    def test_26_exit(self):
        '''If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if ('DES 26wk Full Outcome' in df_entry.Claim_Rate_Type.values):
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_26])
                claim_avail_date = pd.to_datetime(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
                if (exit_date) < (claim_avail_date):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]

                    assert (num == 0), "Exits before 26 full claim, doesn't numerate."
                    assert (den == 0), "Exits before 26 full claim, doesn't denominate."
            
            elif ('DES 26wk Pathway Outcome' in df_entry.Claim_Rate_Type.values):
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Pathway Outcome').argmax()
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_26])
                claim_avail_date = pd.to_date(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
                if (exit_date) < (claim_avail_date):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_path'][0]
                    den = out['Den_26_path'][0]
          
                    assert (num == 0), "Exits before 26 pathway claim, doesn't numerate."
                    assert (den == 0), "Exits before 26 pathway claim, doesn't denominate."

if __name__ == '__main__':
    main = TestNotebook()
    # execute unit test
    import sys

    suite = unittest.TestLoader().loadTestsFromTestCase(TestNotebook)
    unittest.TextTestRunner(verbosity=4, stream=sys.stderr).run(suite)

test_13_date (__main__.TestNotebook)
If 13 week claim available date is BEFORE commencement date -> then 0/0 ... ok
test_13_exit (__main__.TestNotebook)
If a participant exits before 13 week claim is due, then Num=0,Den=0 ... ok
test_13_path (__main__.TestNotebook)
If a 13 week pathway outcome is recorded, then Num=0, Den=1. Participant denominates. ... FAIL
test_26_edu (__main__.TestNotebook)
26 week Education outcomes are not included. Num=0, Den=1 ... ok
test_26_exit (__main__.TestNotebook)
If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0 ... ok
test_26_valid_bonus (__main__.TestNotebook)
26 week Bonus or 26 week work assist outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses ... ok
test_26_valid_full (__main__.TestNotebook)
26 week Full outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses. ... FAIL
test_26_valid_path (__main__.TestNotebook)
26 week Pathway outcomes are included. Num=1, Den=1. Also tests the